# Leetcode中的算法

待解决
- [Longest Palindromic Substring](#5.-Longest-Palindromic-Substring)

并不清楚原理的
- [Beautiful Array](#932.-Beautiful-Array)

怀疑Test case错了的
- [Predict the Winner](#486.-Predict-the-Winner)

新手向教程。因为我自己也是个萌新。以前总想着算法和我无关，遇到问题调用库就完事了。然而最近突然决定打算转CS做码农了。算法变成了不得不补的东西。其实我数学很一般，理解力也很差，所以我自认为我理解的东西，一定可以完全让别人理解（一边写这句话一边开始怀疑了嘻嘻）。我也没学过离散数学（唉～）。现在每天尽量最少做一道题，一点一点进步吧。

这里列一些我认为比较典型的题目

- [动态规划类](#动态规划类总结)
  - [Maximum Subarray](#53.-Maximum-Subarray)
  - [Best Time to Buy and Sell Stock](#121.-Best-Time-to-Buy-and-Sell-Stock)
  - [House Robber](#198.-House-Robber)
  - [Longest Substring Without Repeating Characters](#3.-Longest-Substring-Without-Repeating-Characters)
  - [Minimum Cost Climbing Stairs](#746.-Min-Cost-Climbing-Stairs)
  - [Longest Palindromic Substring](#5.-Longest-Palindromic-Substring)
  - [Maximal Square](#221.-Maximal-Square)
  - [Triangle](#120.-Triangle)
  - [Minimum ASCII Delete Sum for Two Strings](#712.-Minimum-ASCII-Delete-Sum-for-Two-Strings)
  - [Maximum Length of Pair Chain](#646.-Maximum-Length-of-Pair-Chain)
  - [Longest Increasing Subsequence](#300.-Longest-Increasing-Subsequence)
- [数学类](#数学类总结)
  - [Bitwise AND of Numbers Range](#201.-Bitwise-AND-of-Numbers-Range)
  - [Stone Game](#877.-Stone-Game)
  - [Nth Digit](#400.-Nth-Digit)
- [回溯类](#回溯类总结)
  - [Additive Number](#306.-Additive-Number)
- 杂项
  - [1-bit and 2-bit Characters](#717.-1-bit-and-2-bit-Characters)久违地用到了有限状态机

## 11. Container With Most Water

<https://leetcode.com/problems/container-with-most-water/description/>

本质上是找到
$$
    \max\left\{\min\{a_i, a_j\} \cdot (j - i) \middle| 0 \leq i \leq j \leq n - 1\right\}
$$

In [ ]:
class Solution:
    def maxArea(self, height):
        """
        :type height: List[int]
        :rtype: int
        """
        maximum = 0
        left = 0
        right = len(height) - 1
        possibleMaximum = 0
        while left < right:
            possibleMaximum = (right - left) * min(height[left], height[right])
            maximum = max(maximum, possibleMaximum)
            if height[left] <= height[right]:
                left += 1
            else:
                right -= 1
        return maximum

## 53. Maximum Subarray

<https://leetcode.com/problems/maximum-subarray/description/>

本质上是找到
$$
    \max\left\{\sum_{k = i}^{j} a_k \middle| 0 \leq i \leq j \leq n - 1\right\}
$$

用一个叫做[Kadane's algorithm](https://en.wikipedia.org/wiki/Maximum_subarray_problem)的算法。有点数学归纳法的感觉。

以
```
A = -2 1 -3 4 -1 2 1 -5
```
为例，定义$B_i$为`A[0: i+1]`的所有子序列中、所有包含第$i$个元素的子序列的和的最大值。

这个很难看懂。我第一次看也很久才看明白。我们按照严格定义来。首先`A[0: i+1]`的所有“包含第$i$个元素的”子序列可以写成一个集合
$$
\begin{align}
    \left\{A[k, i + 1] \middle| k \in [0, i]\right\}
\end{align}
$$

> 以上面的例子为例，$A[0: 3]$的所有子序列是
```
A[0: 1], A[0: 2], A[0: 3], 
A[1: 2], A[1: 3],
A[2: 3]
```
但是包含“第$i$个元素的”子序列只有
```
A[0: 3], A[1: 3], A[2: 3]
```

$B_i$的定义是
$$
\begin{align}
    B_i = \max\left\{\sum_{m = k}^{i} A_m \middle| k \in [0, i - 1]\right\}
\end{align}
$$

> 以上面的例子为例，$B_3$是
```python
max(sum(A[0: 3]), sum(A[1: 3]), sum(A[2: 3]))
```
而**不是**
```python
max(sum(A[0: 1]), sum(A[0: 2]), sum(A[0: 3]), ..., sum(A[2: 3]))
```

> 以上面的例子为例，可以算出所有的$B_i$
```
A -2  1 -3  4 -1  2  1 -5
B -2  1 -2  4  3  5  6  1
```

最后答案是
$$
    \max\{B_i | i \in [0, n - 1]\}
$$

> 以上面的例子为例，最后答案是6。

In [ ]:
class Solution:
    def maxSubArray(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        maximumSummationSoFar = nums[0]
        possibleMaximumSummation = maximumSummationSoFar
        for i in nums[1: ]:
            maximumSummationSoFar = max(i, i + maximumSummationSoFar)
            possibleMaximumSummation = max(maximumSummationSoFar, possibleMaximumSummation)
        return possibleMaximumSummation

很多人可能会和我一样，第一次错误地认为$B_i$的定义是`A[0: i + 1]`的**所有子序列**的和的最大值，而不是**所以包含了第i个元素的**的子序列的和的最大值。

我后来仔细想了这个问题。对于数学归纳法而言，我们需要这一步推到下一步足够方便。如果按照我们臆想的那个错误定义，我们是很难从`A[0: i+1]`的所有子序列的和的最大值、推到`A[0: i+2]`的所有子序列的和的最大值的。但是我们很容易能从`A[0: i+1]`的所有包含第i个元素的子序列的和的最大值、推导出`A[0: i+2]`的所有包含第i+1个元素的子序列的和的最大值的。他们之间的关系就是一个很简单的
$$
\begin{align}
    B_{i + 1} = \max\{B_{i} + A_{i + 1}, A_{i + 1}\}
\end{align}
$$
关系。

可能有人还是没法理解上面这个式子怎么来的。我再仔细讲一遍。一定要记住，$B_i$一定是`A[0: i+1]`的所有包含了第i个元素的子序列的和的最大值。这是什么意思？意思是$B_i$是一个和，这个和一定含有`A[i]`的成分。否则就不满足定义了。

OK，现在我们有了$B_i$了，怎么从$B_i$推到$B_{i + 1}$呢？同样的，$B_{i + 1}$必须包含`A[i+1]`的成分。这样我们就一个一个地试
```
A[i+1: i+2], A[i: i+2], A[i-1: i+2], ..., A[0: i+2]
```
这时候我们发现了一个问题：我们做了重复劳动了。哪里？你想想你刚才是怎么试出$B_i$的？你也是这样一个一个地试的
```
A[i: i+1], A[i-1: i+1], A[i-2: i+1], ..., A[0: i+1]
```
如果你还是看不出来哪里重复劳动了，我再把$B_{i + 1}$的地方写完整给你看
```
A[i+1: i+2] = 0           + A[i+1]
A[i  : i+2] = A[i  : i+1] + A[i+1]
A[i-1: i+2] = A[i-1: i+1] + A[i+1]
...
A[0  : i+2] = A[0  : i+1] + A[i+1]
              ^^^^^^^^^^^
```
有没有看到重复劳动的地方？我们看到，除了第一项`A[i+1: i+2]`也就是`A[i+1]`之外，其他剩下的子序列都是算$B_i$时用到过的子序列、和`A[i+1]`拼起来的。

## 121. Best Time to Buy and Sell Stock

<https://leetcode.com/problems/best-time-to-buy-and-sell-stock/description/>

本质上是找到
$$
    \max\{a_j - a_i | 0 \leq i \leq j \leq n - 1\}
$$

定义第i天的最大利润是$P_i$。那么这天的利润$P_i$和前面几天的利润有什么关系呢？

- 今天股票价格比昨天低

  那我应该昨天就卖了，所以今天的最大利润和昨天的最大利润一样。$P_i = P_{i - 1}$；
  
- 今天股票价格比昨天高、或者和昨天一样高

  那我应该今天卖。因为今天卖肯定比昨天更赚钱。~~今天的利润比昨天会高$a_i - a_{i - 1}$。~~就是这里错了。如果昨天没有卖怎么办？所以今天的利润不是$P_{i - 1} + a_i - a_{i - 1}$，而是$a_i$减去有史以来见过的最低价$m_i$，即$P_i = a_i - m_i$。所以
$$
    P_i = \max\{P_{i - 1}, a_i - m_i\}
$$

这样答案就是$P_{n - 1}$。

In [1]:
class Solution:
    def maxProfit(self, prices):
        """
        :type prices: List[int]
        :rtype: int
        """
        if not prices:
            return 0
        purchasePrice = prices[0]
        profit = 0
        for value in prices[1: ]:
            purchasePrice = min(value, purchasePrice)
            profit = max(profit, value - purchasePrice)
        return profit

In [3]:
class Solution:
    def maxProfit(self, prices):
        """
        :type prices: List[int]
        :rtype: int
        """
        if not prices:
            return 0
        profits = [0]
        minimumPriceSoFar = prices[0]
        for i in range(len(prices))[1: ]:
            minimumPriceSoFar = min(prices[i], minimumPriceSoFar)
            profits.append(max(profits[i - 1], prices[i] - minimumPriceSoFar))
        return profits[-1]

## 152. Maximum Product Subarray

<https://leetcode.com/problems/maximum-product-subarray/description/>

本质上是要找到
$$
    \max\left\{\prod_{k = i}^{j} a_k \middle| 0 \leq i \leq j \leq n - 1\right\}
$$

定义到包括了第i个元素的最大积为$P_i$。$P_i$和前面的$P_j (j < i)$有什么关系呢？如果做过Maximum Subarray，可能我们很粗略的得出
$$
    P_i = \max\{P_{i - 1} \cdot a_i, a_i\}
$$
但是这时候会隐隐有一种感觉，感觉符号可能会来搅局。一个负号可能就会让最大积一下子变成最小积。仔细想想，上面的这个粗略的猜测有一个致命的问题，就是遇到负数之后，就会抛弃前面的乘积，只会取最新的这个负数。比如
```
2  3  4 -1
```
遇到-1后，$P_i$会马上变成-1。但是如果-1的后面还有负数
```
2  3  4 -1 -1
```
$P_i$会变成1，可是显然按照定义，这是不对的，$P_i$应该等于24。

可能有一些突然，但是我们必须再定义一个$P'_i$，它是包括了第i个元素的最小积。所以完备的$P_i$的定义是
$$
\begin{align}
    P_i &= \max\{P_{i - 1} \cdot a_i, a_i, P'_{i - 1} \cdot a_i\} \\
    P'_i &= \min\{P_{i - 1} \cdot a_i, a_i, P'_{i - 1} \cdot a_i\}
\end{align}
$$

这里$P_i$的定义和之前的Maximum Subarray非常类似，它是**包括了第i个元素成分的**最大积。

所以答案也是$\max\{P_i\}$。

In [7]:
class Solution:
    def maxProduct(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        maximumProducts = [nums[0]]
        minimumProducts = [nums[0]]
        for index, value in enumerate(nums[1: ], 1):
            maximumProducts.append(max(maximumProducts[index - 1] * value, minimumProducts[index - 1] * value, value))
            minimumProducts.append(min(minimumProducts[index - 1] * value, maximumProducts[index - 1] * value, value))
        return max(maximumProducts)

## 198. House Robber

<https://leetcode.com/problems/house-robber/description/>

本质上是要找到
$$
    \max\left\{\sum_{k \in K} a_k \right\}, \quad \text{$K$ is a set such that} \begin{cases}
        K \subseteq \{0, 1, \ldots, n - 1\}, \\
        \forall k_1, k_2 \in K, k_1 \neq k_2, |k_1 - k_2| > 1 \\
    \end{cases}
$$

还是动态规划。定义偷到第i家时，获得的最大利润是$P_i$。那么$P_i$与前面的项的关系是什么呢？是
$$
    P_i = \max\{a_i + P_{i - 2}, P_{i - 1}\}
$$
意思是偷到第i家时，有两种选择
- 要么这家不偷
- 要么偷这家，但是前面一家、也就是第i-1家不能偷，否则就触发警报了

最后答案就是$P_{n - 1}$。

In [2]:
class Solution:
    def rob(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        oddSum = 0
        evenSum = 0
        for index, value in enumerate(nums):
            if index % 2 == 0:
                evenSum = max(oddSum, evenSum + value)
            else:
                oddSum = max(evenSum, oddSum + value)
        return max(oddSum, evenSum)

In [ ]:
class Solution:
    def rob(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        if not nums:
            return 0
        if len(nums) == 1:
            return nums[0]
        maximumSums = [nums[0], max(nums[0], nums[1])]
        for i in range(len(nums))[2: ]:
            maximumSums.append(max(nums[i] + maximumSums[i - 2], maximumSums[i - 1]))
        return maximumSums[-1]

## 213. House Robber II

https://leetcode.com/problems/house-robber-ii/description/

和I不同之处在于，这里的房子围成一个圆圈，所以第一间房子和最后一间房子是相邻的。

总之就是抢了第一家，最后一家不能抢；或者抢了最后一家，第一家就不能抢；也可以第一家和最后一家都不抢。

一种方法是直接I的程序跑三遍。取最大值好了。

In [150]:
class Solution:
    def rob(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        if not nums:
            return 0
        if len(nums) == 1:
            return nums[0]
        if len(nums) == 2 or len(nums) == 3:
            return max(nums)
        q = []
        for nums in [nums[0: -1], nums[1: ], nums[1: -1]]:
            maximumSums = [nums[0], max(nums[0], nums[1])]
            for i in range(len(nums))[2: ]:
                maximumSums.append(max(nums[i] + maximumSums[i - 2], maximumSums[i - 1]))
            q.append(maximumSums[-1])
        return max(q)

In [151]:
Solution().rob([2,3,2])

3

In [152]:
Solution().rob([1,2,3,1])

4

## 122. Best Time to Buy and Sell Stock II

<https://leetcode.com/problems/best-time-to-buy-and-sell-stock-ii/description/>

本质上是要找到
$$
    \max\left\{\sum_{k=1}^N (a_{s_k} - a_{b_k}) \middle| 0 \leq b_1 < s_1 < b_2 < s_2 < \ldots < b_N < s_N \leq n - 1\right\}
$$

这题和上一题的区别是Best Time to Buy and Sell Stock只能交易一次。这里是可以交易无限次，但是限制是每天只能最多做一次交易，并且手里不能同时有两只股票。也就是说不能第一天买，第二天又买，再都第三天卖掉。

设第i天最大利润是$P_i$，那么$P_{i + 1}$和$P_i$有什么关系？如果第i+1天价格$a_{i + 1}$大于第i天价格$a_i$，那说明昨天不应该卖，应该今天卖掉，这样总利润比昨天高$a_{i + 1} - a_i$；如果今天价格比昨天低或者一样，那么昨天一定要卖掉，今天再入手抄底，这样总利润和昨天一样，不变。
$$
\begin{align}
    P_{i + 1} = \begin{cases}
        P_i + a_{i + 1} - a_i, &\qquad a_{i + 1} > a_i \\
        P_i &\qquad a_{i + 1} \leq a_i \\
    \end{cases}
\end{align}
$$

所以答案是原来数列的一阶差分、再过滤掉所有负数项、之后的和。
$$
    \sum_{i = 0}^{n - 2} \max\{a_{i + 1} - a_i, 0\}
$$

In [26]:
class Solution:
    def maxProfit(self, prices):
        """
        :type prices: List[int]
        :rtype: int
        """
        return sum(prices[i + 1] - prices[i] if prices[i + 1] > prices[i] else 0 for i in range(len(prices) - 1))

In [27]:
Solution().maxProfit([1,2,3,4,5])

4

## 3. Longest Substring Without Repeating Characters

<https://leetcode.com/problems/longest-substring-without-repeating-characters/description/>

本质上是要找到
$$
    \max\left\{j - i \middle| \forall k_1, k_2 \in [i, j], k_1 \neq k_2: a_{k_1} \neq a_{k_2}\right\}
$$

定义$S_i$为包含了$a_i$的、0到$a_i$之间最长的无重复字符的子字符串。那么$S_{i + 1}$和$S_i$有什么关系呢？
- 如果$a_{i + 1}$不在$S_i$里，那么$S_{i + 1} = \{S_i, a_{i + 1}\}$
- 如果$S_i$里面已经有$a_{i + 1}$了，$S_i$的定义可以保证$S_i$有且只有一次$a_{i + 1}$。找到它出现的地方，从它后面一个字符开始、把$S_i$切开成两段，取后面一段$S'_i$，这样$S_{i + 1} = \{S'_i, a_{i + 1}\}$

答案就是$\max\{S_i\}$。

In [9]:
class Solution:
    def lengthOfLongestSubstring(self, s):
        """
        :type s: str
        :rtype: int
        """
        if len(s) == 0:
            return 0
        if len(s) == 1:
            return 1
        longestSubstringContainingCurrentCharacter = s[0]
        longestSubstringLength = 1
        for value in s[1: ]:
            if value in longestSubstringContainingCurrentCharacter:
                longestSubstringContainingCurrentCharacter = longestSubstringContainingCurrentCharacter[longestSubstringContainingCurrentCharacter.index(value) + 1: ] + value
                longestSubstringLength = max(longestSubstringLength, len(longestSubstringContainingCurrentCharacter))
            else:
                longestSubstringContainingCurrentCharacter = longestSubstringContainingCurrentCharacter + value
                longestSubstringLength = max(longestSubstringLength, len(longestSubstringContainingCurrentCharacter))
        return longestSubstringLength

In [13]:
Solution().lengthOfLongestSubstring("abcabcbb")

3

## 123. Best Time to Buy and Sell Stock III

<https://leetcode.com/problems/best-time-to-buy-and-sell-stock-iii/description/>

这题和II的区别是，最多交易2次。

本质上是找到
$$
    \max\{\underbrace{\max\{a_j - a_i + a_l - a_k | 0 \leq i < j < k < l \leq n - 1\}}_{交易2次}, \underbrace{\max\{a_j - a_i | 0 \leq i < j \leq n - 1\}}_{交易1次}, \underbrace{0}_{不交易}\}
$$

In [ ]:
class Solution:
    def maxProfit(self, prices):
        """
        :type prices: List[int]
        :rtype: int
        """
        

## 70. Climbing Stairs

<https://leetcode.com/problems/climbing-stairs/description/>

到达第$i$级楼梯的方法有2种
- 从第$i - 1$级跨1级上来
- 从第$i - 2$级跨2级上来

设$p_i$是达到第$i$级的方法数量。那么$p_i$和前面的项有什么关系呢？
$$
    p_i = p_{i - 1} + p_{i - 2}
$$

再想初始条件，如果一个楼梯只有1级，那当然只有一种方法，即$p_1 = 1$。如果一个楼梯有2级，那么有两种方法
- 直接跨到第2级
- 从第1级跨到第2级

即$p_2 = 2$。

完整的递推式就写出来了
$$
\left\{\begin{align}
    p_i &= p_{i - 1} + p_{i - 2} \\
    p_1 &= 1 \\
    p_2 &= 2
\end{align}\right.
$$

最好想的算法就出来了。

In [ ]:
class Solution:
    def climbStairs(self, n):
        """
        :type n: int
        :rtype: int
        """
        if n == 1:
            return 1
        if n == 2:
            return 2
        q = [0, 1, 2]
        for i in range(3, n + 1):
            q.append(q[i - 1] + q[i - 2])
        return q[-1]

当然也可以用数学方法降维打击。这个显然是个位移过的斐波那契数列`1 2 3 5 8 ...`。

## 746. Min Cost Climbing Stairs

<https://leetcode.com/problems/min-cost-climbing-stairs/description/>

~~假设到了第$i - 1$级楼梯，此时最小成本是$P_{i - 1}$。那么$P_i$和$P_{i - 1}$有什么关系呢？~~

~~如果第$i - 1$级楼梯是被略过的，那么不能跨过超过2级楼梯，第$i$级楼梯必须上去，所以$P_i = P_{i - 1} + a_i$。~~

~~如果第$i - 1$级楼梯没有被略过，是花了成本的，那么可以比较一下$a_i$和$a_{i - 1}$，如果~~
- ~~发现$a_i$比$a_{i - 1}$小，那说明其实第$i - 1$级刚才应该跳过的，应该直接从第$i - 2$级跳到第$i$级~~
- ~~发现$a_i$比$a_{i - 1}$大，那么还是算了，第$i$级不要上去了吧，观望一下~~
- ~~发现$a_i = a_{i - 1}$，这就有点难处理了。但是根据贪心的原则，还是应该到第$i$级上去。~~

很可惜，上面我想了这么多，最后还是没能做出来。之前也做了这么多道DP题，遇到这种题还是做不出。这里也算是反面教材了一下。事实上DP是很简单的。只要想一个核心问题，就是怎样从前面的条件，比如$i - 1, i - 2$推到$i$，不需要考虑$i + 1$的情况。

怎样到达第$i$级楼梯呢？我们发现只有两种方法
- 从第$i - 2$级跨2级上来
- 从第$i - 1$级跨1级上来

这两种方法里肯定有一种成本最低。那就不管了，直接用一个$\min$套起来
$$
    p_i = \min\{p_{i - 2} + a_{i - 2}, p_{i - 1} + a_{i - 1}\}
$$
这时候千万不要想$i + 1$怎么样，完全不需要想。

注意这个题目里面的$a_i$不是指从到达第$i$级的成本，而是指**从第$i$级出发、上一级（到达$i + 1$）或者两级（到达$i + 2$）的成本。**所以自然$p_i$也是指的到达第$i$级花的成本，里面是没有$a_i$的成分的。如果你要从第$i$级出发，才需要加上$a_i$。

这时候再去想初始条件。因为一开始可以选择直接跨上第0个台阶，也可以选择直接跨上第1个台阶。**是不需要成本的！**这个条件非常关键，否则你就会写成`p[0] = cost[0], p[1] = min(cost[0], cost[1]）`。

所以完整的递推式是
$$
\left\{\begin{align}
    p_i &= \min\{p_{i - 2} + a_{i - 2}, p_{i - 1} + a_{i - 2}\} \\
    p_0 &= 0 \\
    p_1 &= 0 \\
\end{align}\right.
$$

最终答案不是$p_{n - 1}$而是$p_n$。可以这么理解，最后你要跳到顶层，可以把顶层看成成本为0的一级楼梯。

In [124]:
class Solution:
    def minCostClimbingStairs(self, cost):
        """
        :type cost: List[int]
        :rtype: int
        """
        p = [0, 0]
        for index, value in enumerate(cost[2: ] + [0], 2):
            p.append(min(p[index - 2] + cost[index - 2], p[index - 1] + cost[index - 1]))
        print(p)
        return p[-1]

In [125]:
Solution().minCostClimbingStairs([10, 15, 20])

[0, 0, 10, 15]


15

In [126]:
Solution().minCostClimbingStairs([1, 100, 1, 1, 1, 100, 1, 1, 100, 1])

[0, 0, 1, 2, 2, 3, 3, 4, 4, 5, 6]


6

In [127]:
Solution().minCostClimbingStairs([0, 0, 0, 1])

[0, 0, 0, 0, 0]


0

## 279. Perfect Squares

https://leetcode.com/problems/perfect-squares/description/

本质上是把一个数$m$表示成
$$
    m = \sum_{k = 1}^{+\infty} a_k \cdot k^2
$$
然后找到
$$
    \min\left\{\sum_{k = 1}^{+\infty} a_k\right\}
$$

找了下资料，发现这个是一个叫[四平方定理 Lagrange's four-square theorem](https://en.wikipedia.org/wiki/Lagrange%27s_four-square_theorem)的东西。意思是任何一个数都可以表示成
$$
    m = a_0^2 + a_1^2 + a_2^2 + a_3^2
$$
其中$a_0, a_1, a_2, a_3 \in \mathbb{N}$，并且可以相等。

大致思路是先找到比$m$小的最大的平方数$k$。

In [ ]:
class Solution:
    def numSquares(self, n):
        """
        :type n: int
        :rtype: int
        """
        

## 32. Longest Valid Parentheses

https://leetcode.com/problems/longest-valid-parentheses/description/

设$p_i$是包括了第$i$个字符的最长的valid parenthese字符串长度。那么$p_i$和前面的项有什么关系呢？



In [ ]:
class Solution:
    def longestValidParentheses(self, s):
        """
        :type s: str
        :rtype: int
        """
        

## 70. Climbing Stairs

<https://leetcode.com/problems/climbing-stairs/description/>

好像又是动态规划。设$f_n$是n级楼梯的爬法数量，需要找到$f_{n + 1}$和$f_n$的关系。

发现爬到第n+1级有两种办法
- 从第n级爬1级到第n+1级
- 从第n-1级爬2级到第n+1级

可能有人有疑问那么为什么不可以从第n-1级爬1级、再爬1级到第n+1级呢？因为这个方案已经被**从第n级爬1级到第n+1级**包括了。

所以得到了$f_{n + 1}$的表达式
$$
\begin{align}
    f_{n + 1} = f_n + f_{n - 1}
\end{align}
$$
所以可以用动态规划遍历一遍，就得到答案了。

还可以用数学方法降维打击。这个看上去有点像斐波那契数列。但是这个数列从1开始，也就是`1, 2, 3, 5 ...`而不是`1, 1, 2, 3, 5`。

斐波那契数列的通项公式是
$$
    F_n = {\left({1 + \sqrt{5} \over 2}\right)^n - \left({1 - \sqrt{5} \over 2}\right)^n \over \sqrt{5}}
$$
在我们这个题目里面，$f_n = F_{n + 1}$。

为了防止小数运算出错，外面再加一个`round`四舍五入到整数。

数学方法

In [19]:
from math import sqrt
class Solution:
    def climbStairs(self, n):
        """
        :type n: int
        :rtype: int
        """
        return round((((1 + sqrt(5)) / 2) ** (n + 1) - ((1 - sqrt(5)) / 2) ** (n + 1)) / sqrt(5))

In [12]:
Solution().climbStairs(5)

8

动态规划做法。

In [20]:
class Solution:
    def climbStairs(self, n):
        """
        :type n: int
        :rtype: int
        """
        if n == 1:
            return 1
        if n == 2:
            return 2
        q = [0, 1, 2]
        for i in range(3, n + 1):
            q.append(q[i - 1] + q[i - 2])
        return q[-1]

In [25]:
Solution().climbStairs(5)

8

## 221. Maximal Square

https://leetcode.com/problems/maximal-square/description/

类似卡诺图问题。需要在图中找到一个面积为$n^2$的最大的方块区域。

这个题居然也是动态规划题。我是实在没有想到。说明我对动态规划的理解还是有欠缺的。以前的动态规划题，$\{p_i\}$都是一维的数组。其实多维的也是可以的。比如这里可以设$p_{i, j}$是以点$(i, j)$为最右下角的、最大正方形的边长。那么$p_{i, j}$和前面的项有什么关系呢？

注意这里因为是二维的情况，此处说的“前面的项”，不仅包括了$p_{i - 1, j - 1}$，也包括$p_{i - 1, j}, p_{i, j - 1}$等等。

> 这个有点类似二维随机概率分布函数的算法。记得一维概率分布函数的算法是$\int_{- \infty}^x f(x) \,dx$，二维是$\int_{- \infty}^x \int_{- \infty}^y f(x, y) \,dx dy$。

$$
\left\{\begin{align}
    p_{i, j} &= \begin{cases}
        \min\{p_{i - 1, j - 1}, p_{i - 1, j}, p_{i, j - 1}\} + 1, &\qquad a_{i, j} = 1 \\
        0, &\qquad a_{i, j} = 0
    \end{cases} \\
    p_{0, j} &= a_{0, j}, \qquad \forall j \in [0, n - 1] && 上边界\\
    p_{i, 0} &= a_{i, 0}, \qquad \forall i \in [0, m - 1] && 左边界
\end{align}\right.
$$

In [434]:
import copy
class Solution(object):
    def maximalSquare(self, matrix):
        """
        :type matrix: List[List[str]]
        :rtype: int
        """
        if matrix == [] or matrix == [[]]:
            return 0
        p = copy.deepcopy(matrix)
        p[0] = [int(i) for i in p[0]]
        for j in range(1, len(matrix)):
            p[j][0] = int(p[j][0])
            
        for i in range(1, len(matrix)):
            for j in range(1, len(matrix[0])):
                p[i][j] = min(p[i - 1][j - 1], p[i][j - 1], p[i - 1][j]) + 1 if int(matrix[i][j]) == 1 else 0
#         print(p)
        return max(max(i) for i in p) ** 2

In [435]:
Solution().maximalSquare([
[1, 0, 1, 0, 0,],
[1, 0, 1, 1, 1,],
[1, 1, 1, 1, 1,],
[1, 0, 0, 1, 0,],
])

2

## 264. Ugly Number II

https://leetcode.com/problems/ugly-number-ii

设第i个ulgy number是$p_i$。

In [494]:
class Solution:
    def nthUglyNumber(self, n):
        """
        :type n: int
        :rtype: int
        """
        if n == 1:
            return 1
        t2 = 0
        t3 = 0
        t5 = 0
        p = [1]
        for i in range(1, n):
            p.append(min(p[t2] * 2, p[t3] * 3, p[t5] * 5))
            if p[-1] == p[t2] * 2:
                t2 += 1
            if p[-1] == p[t3] * 3:
                t3 += 1
            if p[-1] == p[t5] * 5:
                t5 += 1
                
#         print(p)
        return p[-1]

In [495]:
Solution().nthUglyNumber(7)

[1, 2, 3, 4, 5, 6, 8]


8

## 396. Rotate Function

https://leetcode.com/problems/rotate-function/description/

假设
$$
    A = (\begin{matrix}
    a_0 & a_1 & a_2 & \ldots & a_{n - 1}
    \end{matrix})^T
$$
要找到
$$
    \left(\begin{matrix}
        0 & 1 & 2 & \ldots & n - 2 & n - 1 \\
        1 & 2 & 3 & \ldots & n - 1 & 0 \\
        2 & 3 & 4 & \ldots & 0 & 1 \\
        \vdots & & & \ddots & & \vdots \\
        n - 1 & 0 & 1 & \ldots & n - 3 & n - 2 \\
    \end{matrix}\right)\left(\begin{matrix}
        a_0 \\
        a_1 \\
        a_2 \\
        \vdots \\
        a_{n - 1}
    \end{matrix}\right) = B
$$
中最大的元素。

但其实上面的式子不能告诉我们什么。观察$b_i$和$b_{i - 1}$的关系。
$$
    b_i - b_{i - 1} = \left(\sum_{k = 0}^{n - 1} a_k\right) - n a_{n - i}
$$
初始条件
$$
\begin{align}
    b_0 &= 0 a_0 + 1 a_1 + 2 a_2 + \ldots + (n - 1) a_{n - 1} \\
    &= \sum_{k = 0}^{n - 1} k a_k
\end{align}
$$

In [548]:
class Solution:
    def maxRotateFunction(self, A):
        """
        :type A: List[int]
        :rtype: int
        """
        if len(A) == 0 or len(A) == 1:
            return 0
        aSum = sum(A)
        aLength = len(A)
        b = [sum(i * A[i] for i in range(aLength))]
        for index, value in enumerate(A[1: ], 1):
            b.append(b[-1] + aSum - aLength * A[aLength - index])
        return max(b)

In [549]:
Solution().maxRotateFunction([4, 3, 2, 6])

26

## 363. Max Sum of Rectangle No Larger Than K

https://leetcode.com/problems/max-sum-of-rectangle-no-larger-than-k/description/

和[Maximal Square](#221.-Maximal-Square)有点像。

设$p_{i, j}$是以$(i, j)$为右下角的~~正方形的~~看错题了，这里应该是长方形、不超过$k$的最大和。那么$p_{i, j}$和前面的项有什么关系呢？
$$
    p_{i, j} = 
$$

算了，这个真的没法做。

In [567]:
import copy
class Solution:
    def maxSumSubmatrix(self, matrix, k):
        """
        :type matrix: List[List[int]]
        :type k: int
        :rtype: int
        """
        if matrix == [] or matrix == [[]]:
            return 0
        
        p = copy.deepcopy(matrix)
        
        for i in range(len(matrix)):
            for j in range(len(matrix[0])):
                if i == 0 and j == 0: # upper left corner
                    p[i][j] = max(filter(lambda x: x <= k, [matrix[i][j]]), default=0)
                elif i == 0:
                    p[i][j] = max(filter(lambda x: x <= k, [p[i][j - 1] + matrix[i][j], 
                                                            matrix[i][j]]), default=0)
                elif j == 0:
                    p[i][j] = max(filter(lambda x: x <= k, [p[i - 1][j] + matrix[i][j],
                                                            matrix[i][j]]), default=0)
                else:
                    p[i][j] = max(filter(lambda x: x <= k, [p[i - 1][j - 1] + matrix[i][j], 
                                                            p[i][j - 1] + matrix[i][j], 
                                                            p[i - 1][j] + matrix[i][j],
                                                            matrix[i][j]]), default=0)
        print(p)
        return max(max(i) for i in p)

In [563]:
Solution().maxSumSubmatrix([[1,0,1],[0,-2,3]], 2)

2

In [564]:
Solution().maxSumSubmatrix([[3, 3, 3],[3, 3, 3]], 2)

0

In [568]:
Solution().maxSumSubmatrix([[2, 2, -1]], 3)

[[2, 2, 1]]


2

## 368. Largest Divisible Subset

https://leetcode.com/problems/largest-divisible-subset/description/

In [ ]:
class Solution:
    def largestDivisibleSubset(self, nums):
        """
        :type nums: List[int]
        :rtype: List[int]
        """
        

## 5. Longest Palindromic Substring

https://leetcode.com/problems/longest-palindromic-substring/description/

找到最长的回文子字符串。

In [125]:
class Solution:
    def longestPalindrome(self, s):
        """
        :type s: str
        :rtype: str
        """
        pool = set()
        for i in range(len(s)):
            for j in range(i + 1, len(s) + 1):
                if s[i: j] == s[i: j][:: -1]:
                    pool.add(s[i: j])
                    
        return max(pool, key=lambda x: len(x), default="")

In [126]:
Solution().longestPalindrome("banana")

'anana'

我一开始只能想到上面的暴力算法，但是这个会time limit exceed。

有一种叫Manacher的算法。可以优化到$O(n)$复杂度。

一些资料
- https://segmentfault.com/a/1190000008484167
- https://en.wikipedia.org/wiki/Longest_palindromic_substring

下面的代码是segmentfault上直接抄过来的。

In [127]:
class Solution:
    def longestPalindrome(self, s):
        """
        :type s: str
        :rtype: str
        """
        s = '#' + '#'.join(s) + '#'
        l = len(s)
        r = [0] * l
        mx, pos = 0, 0
        for i in range(l):
            if i > mx:
                r[i] = 1
            else:
                r[i] = min(r[2 * pos - i], mx - i)

            while i - r[i] >= 0 and i + r[i] < l and s[i + r[i]] == s[i - r[i]]:
                r[i] += 1

            if r[i] + i - 1 > mx:
                mx = r[i] + i - 1
                pos = i
        radius = max(r) - 1
        s1 = r.index(radius + 1) - radius
        s2 = s1 + radius * 2
        return s[s1: s2].replace("#", "")

In [124]:
Solution().longestPalindrome("banana")

'anana'

设$p_i$是以$a'_i$为中心的最长回文字符串的**半径**。但是其实这里叫半径不是特别贴切，比如
```
i 0 1 2 3 4 5 6
---------------
  # a # b # b #
---------------
p 1 2 1 2 3 2 1
```
中，$p_0$是1，$p_1$是2。只能说这里的半径的定义是回文字符串总长度的一半减0.5。

那么可以看到最长回文子字符串的长度就是$\max\{p_i\} - 1$。再通过$\operatorname{argmax}$转换一下就可以得到那个最长回文子字符串了。

In [42]:
import operator
class Solution:
    def longestPalindrome(self, s):
        """
        :type s: str
        :rtype: str
        """
        s = '#' + '#'.join(s) + '#'
        dp = [1]
        for i in range(1, len(s)):
            radius = 1
            for j in range(1, min(i, len(s) - i - 1) + 1): # 从中心开始往两边逐个检查
                if s[i - j] == s[i + j]: # 遇到一样的
                    radius += 1
                else: # 遇到不一样的
                    break
            dp.append(radius)
            
        argmaxRadius, maxRadius = max(enumerate(dp), key=operator.itemgetter(1)) # 得到最长回文子字符串的中心下标和半径
        s1 = argmaxRadius - maxRadius + 1 # 左边界
        s2 = argmaxRadius + maxRadius # 右边界
        return s[s1: s2].replace("#", "")

In [45]:
Solution().longestPalindrome("banana")

'anana'

稍微快一点的版本。

In [46]:
class Solution:
    def longestPalindrome(self, s):
        """
        :type s: str
        :rtype: str
        """
        s = '#' + '#'.join(s) + '#'
        argmaxRadius = 0
        maxRadius = 1
        for i in range(1, len(s)):
            radius = 1
            for j in range(1, min(i, len(s) - i - 1) + 1): # 从中心开始往两边逐个检查
                if s[i - j] == s[i + j]: # 遇到一样的
                    radius += 1
                else: # 遇到不一样的
                    break
            if radius > maxRadius:
                maxRadius = radius
                argmaxRadius = i
            
        s1 = argmaxRadius - maxRadius + 1 # 左边界
        s2 = argmaxRadius + maxRadius # 右边界
        return s[s1: s2].replace("#", "")

In [47]:
Solution().longestPalindrome("banana")

'anana'

上面的代码根本没有用到dp的思想，而且复杂度应该还是$O(n^2)$。但是却可以过。不过这样就没有意思了。

还是来思考一下递推式。

In [56]:
class Solution:
    def longestPalindrome(self, s):
        """
        :type s: str
        :rtype: str
        """
        s = '#' + '#'.join(s) + '#'
        dp = [1]
        argmaxRadius = 0
        maxRadius = 1
        for i in range(1, len(s)):
            if i < argmaxRadius + maxRadius:
                radius = dp[2 * argmaxRadius - i]
            else:
                radius = 1
            searchRange = range(radius, min(i, len(s) - i - 1) + 1)
            for j in searchRange: # 从中心开始往两边逐个检查
                if s[i - j] == s[i + j]: # 遇到一样的
                    radius += 1
                else: # 遇到不一样的
                    break
            dp.append(radius)
            if radius > maxRadius:
                maxRadius = radius
                argmaxRadius = i
            
        s1 = argmaxRadius - maxRadius + 1 # 左边界
        s2 = argmaxRadius + maxRadius # 右边界
        return s[s1: s2].replace("#", "")

In [57]:
Solution().longestPalindrome("banana")

'anana'

## 516. Longest Palindromic Subsequence

这道题和上面的longest palindromic substring的区别在于，这道题里面不要求连续。

## 898. Bitwise ORs of Subarrays

https://leetcode.com/problems/bitwise-ors-of-subarrays/description/

本质上是要找到
$$
    \left\{\prod_{k = i}^j a_k \middle| 0 \leq i \leq j \leq n - 1\right\}
$$

In [212]:
import functools
class Solution:
    def subarrayBitwiseORs(self, A):
        """
        :type A: List[int]
        :rtype: int
        """
        if A == []:
            return 0
        p = [{A[0]}]
        for index, value in enumerate(A[1: ], 1):
            p.append({i | value for i in p[-1]} | {value, })
#         print(p)
        return len(functools.reduce(lambda x, y: x.union(y), p))

上面是暴力解法。肯定会time limit exceed。

In [195]:
class Solution:
    def subarrayBitwiseORs(self, A):
        """
        :type A: List[int]
        :rtype: int
        """
        if A == []:
            return 0
        p = [1]
        allOrs = {A[0]}
        lastOr = A[0]
        for index, value in enumerate(A[1: ], 1):
            currentOr = lastOr | value
            allOrs.update([value, currentOr, value | A[index - 1]])
            
            lastOr = currentOr
#         print(allOrs)
        return len(allOrs)

In [213]:
Solution().subarrayBitwiseORs([0])

[{0}]


1

In [214]:
Solution().subarrayBitwiseORs([1, 1, 2])

[{1}, {1}, {2, 3}]


3

In [215]:
Solution().subarrayBitwiseORs([1, 2, 4])

[{1}, {2, 3}, {4, 6, 7}]


6

In [216]:
Solution().subarrayBitwiseORs([3, 11])

[{3}, {11}]


2

In [217]:
Solution().subarrayBitwiseORs([30, 126, 88, 97, 98, 66])

[{30}, {126}, {88, 126}, {121, 97, 127}, {99, 98, 123, 127}, {98, 99, 66, 123, 127}]


10

## 650. 2 Keys Keyboard

https://leetcode.com/problems/2-keys-keyboard/description/

设打到$i$个字符时，需要的最少步骤数量是$P_i$。那么$P_i$和前面的项有什么关系呢？

如果$i$有因数$a_1, a_2, \ldots, a_n$，那么就有$n$种方法构成$i$个A
- $a_1$个A复制1下，然后粘贴$i / a_1 - 1$次，所以总共在$P_{a_1}$的基础上再操作$i / a_1$次
- $a_2$个A复制1下，然后粘贴$i / a_2 - 1$次
- ...
- $a_n$个A复制1下，然后粘贴$i / a_n - 1$次

也就是说
$$
    P_i = \min\left\{P_{a_k} + {i \over a_k} \middle| k \in \{1, 2, \ldots, n\}\right\}
$$

初始条件
$$
    P_1 = 0
$$

In [304]:
import math
class Solution:
    def minSteps(self, n):
        """
        :type n: int
        :rtype: int
        """
        if n == 1:
            return 0
        p = [0, 0]
        for i in range(2, n + 1):
            divisors = []
            for divisor in range(1, i // 2 + 1):
                if i % divisor == 0:
                    divisors.append(divisor)
#             print(i, divisors)
            p.append(min([p[ak] + i // ak for ak in divisors]))
#         print(p)
        return p[-1]

In [305]:
Solution().minSteps(9)

6

最快的好像是用递归做的，但是python限制递归深度。所以我不推荐用。

下面这种也不错。

In [303]:
class Solution:
    def minSteps(self, n):
        """
        :type n: int
        :rtype: int
        """
        ans = 0
        d = 2
        while n > 1:
            while n % d == 0:
                ans += d
                n /= d
            d += 1
        return ans

## 120. Triangle

找到三角形从上到下最小花费。

设$p_{i, j}$是到达第i行、第j个元素的最小花费。

In [22]:
class Solution:
    def minimumTotal(self, triangle):
        """
        :type triangle: List[List[int]]
        :rtype: int
        """
        if len(triangle) == 0:
            return 0
        p = [[triangle[0][0]]]
        for index, value in enumerate(triangle[1: ], 1):
            thisRow = [p[-1][0] + value[0]]
            for i, v in enumerate(value[1: -1], 1):
                thisRow.append(min(p[-1][i - 1], p[-1][i]) + v)
            thisRow.append(p[-1][-1] + value[-1])
#             print(thisRow)
            p.append(thisRow)
        return min(p[-1])

稍微优化点空间。

突然发现不仅优化了空间，还顺便优化了时间。可能因为python列表的`.append()`很慢吧。

In [25]:
class Solution:
    def minimumTotal(self, triangle):
        """
        :type triangle: List[List[int]]
        :rtype: int
        """
        if len(triangle) == 0:
            return 0
        p = [triangle[0][0]]
        for index, value in enumerate(triangle[1: ], 1):
            thisRow = [p[0] + value[0]]
            for i, v in enumerate(value[1: -1], 1):
                thisRow.append(min(p[i - 1], p[i]) + v)
            thisRow.append(p[-1] + value[-1])
#             print(thisRow)
            p = thisRow
        return min(p)

还有一种更好的办法是从底往上考虑。这样做有2个好处
- 最终答案就是`p[0]`，不需要再费$O(n)$时间算出`min(p)`
- 不用单独考虑最边上的元素，代码更好看

In [7]:
class Solution:
    def minimumTotal(self, triangle):
        """
        :type triangle: List[List[int]]
        :rtype: int
        """
        if len(triangle) == 0:
            return 0
        p = triangle[-1]
        for index, value in enumerate(triangle[: -1][:: -1]):
            p = [min(p[i], p[i + 1]) + value[i] for i in range(len(value))]
        return p[0]

In [8]:
Solution().minimumTotal([
     [2],
    [3,4],
   [6,5,7],
  [4,1,8,3]
])

11

In [9]:
Solution().minimumTotal([[-1]
                        ,[2,3]
                      ,[1,-1,-3]])

-1

## 931. Minimum Falling Path Sum

感觉和上一题triangle差不多。还是用DP的方法生成到当前这一行每个元素的最小花费。

In [10]:
class Solution:
    def minFallingPathSum(self, A):
        """
        :type A: List[List[int]]
        :rtype: int
        """
        if A == [] or A == [[]]:
            return 0
        if len(A) == 1:
            return A[0][0]
        lastRowCost = A[0]
        for index, row in enumerate(A[1: ], 1):
            leftCost = [row[0] + min(lastRowCost[0], lastRowCost[1])]
            rightCost = [row[-1] + min(lastRowCost[-1], lastRowCost[-2])]
            middleCost = []
            for i, value in enumerate(row[1: -1], 1):
                middleCost.append(value + min(lastRowCost[i - 1: i + 2]))
            lastRowCost = leftCost + middleCost + rightCost
        return min(lastRowCost)

In [11]:
Solution().minFallingPathSum([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

12

In [13]:
Solution().minFallingPathSum([
    [1, 2],
    [3, 4]
])

4

## 413. Arithmetic Slices

有点maximum subarray的变种的感觉。感觉凡是遇到这种**最长连续子序列**的问题都可以这么做。

用$p_i$表示，以第i个元素为结尾的、长度大于等于3的数列切片的数量。那么$p_i$和前面的项有什么关系？

发现，因为$p_{i - 1}$是以前面一项为结尾的、长度大于等于3的数列切片的数量，所以如果
- $a_i$和$a_{i - 1}, a_{i - 2}$能构成等差数列，那么$p_i = p_{i - 1} + 1$
- $a_i$和$a_{i - 1}, a_{i - 2}$不构成等差数列，那么$p_i = 0$

In [6]:
class Solution:
    def numberOfArithmeticSlices(self, A):
        """
        :type A: List[int]
        :rtype: int
        """
        if len(A) < 3:
            return 0
        if A[1] * 2 == A[0] + A[2]:
            p = [0, 0, 1]
            q = 1
        else:
            p = [0, 0, 0]
            q = 0
        for index, value in enumerate(A[3: ], 3):
            if A[index - 1] * 2 == A[index - 2] + value:
                p.append(p[-1] + 1)
                q += p[-1]
            else:
                p.append(0)
#         return sum(q) # 最后的sum很慢。改成一边加一边算，最后就不用sum了，比较快。
        return q

In [7]:
Solution().numberOfArithmeticSlices([1, 2, 3, 4])

3

## 712. Minimum ASCII Delete Sum for Two Strings

粗看，感觉像是
1. 找到最长公共子序列（不一定连续）
1. 各自删掉不在最长公共子序列里的元素

但是有个问题，保留最长公共子序列不一定是最省钱的方法，因为每个字母的价格不同，它们是有权重的。

怎么从dp的角度思考这个问题呢？

令$p_{i, j}$是使`s1[0: i]`和`s2[0: j]`相等的最小花费。那么$p_{i, j}$和前面的项有什么关系呢？
- 如果`s1[i] == s2[j]`，那么不需要删掉任何一个字符，所以$p_{i, j} = p_{i - 1, j - 1}$
- 如果`s1[i] != s2[j]`，那么说明出现了不相等的字符，一定要删掉某个字符，那么到底删`s1[i]`还是`s2[j]`?
  
  - 如果只删`s1[i]`，那么总花费是$p_{i - 1, j} + \operatorname{ord} s_{1(i)}$
  - 如果只删`s2[j]`，那么总花费是$p_{i, j - 1} + \operatorname{ord} s_{2(j)}$
  - 如果删除`s1[i]`和`s2[j]`，那么总花费是$p_{i - 1, j - 1} + \operatorname{ord} s_{1(i)} + \operatorname{ord} s_{2(j)}$
  
    实际上，这种情况已经被前两种情况包括了。
    
    例子
    ```
    p  |e    e
    --------------
    e  |0    e
    a  |a    ae
    ```
    最右下角的一格的`ae`，可以由第一第二条情况得到，当然第三条也能得到`ae`，但是没必要。
  
再来看一下初始条件。这里要当心一种情况，就是其中一个字符串是空字符串的情况。如果按传统的方法，如果其中一个字符串是空字符串的话，整个表格的就没有面积了。为了处理这种情况，我们把字符串起始位看做单独的一个字符∅。当然这种方法还有更大的一个好处，就是能帮助我们更容易看清楚初始条件。

我们发现，更新第一行的时候，因为s1还是空字符串，所以s2来一个字符，就要删掉一个字符，这样第一行变成了s2的一个累积
```
    ∅    s    e    a
---------------------
∅  |0    s    se   sea
```

更新第一列的时候，也是相同的情况。

所以总结下来，递推式是
$$
\left\{\begin{aligned}
    p_{i, j} &= \min\{p_{i - 1, j} + \operatorname{ord} s_{1(i)}, p_{i, j - 1} + \operatorname{ord} s_{2(j)}\} \\
    p_{0, j} &= \sum_{j = 0}^{\operatorname{len} S_2} \operatorname{ord} s_{(2), j} \\
    p_{i, 0} &= \sum_{j = 0}^{\operatorname{len} S_1} \operatorname{ord} s_{(1), i} \\
\end{aligned}\right.
$$
  
例子
```
    ∅    s    e    a
---------------------
∅  |0    s    se   sea
e  |e    se   s    sa
a  |ea   s    sa   s
t  |eat  seat sat  st
```

In [38]:
import itertools
class Solution:
    def minimumDeleteSum(self, s1, s2):
        """
        :type s1: str
        :type s2: str
        :rtype: int
        """
        s1 = "\x00" + s1
        s2 = "\x00" + s2
        dp = [list(itertools.accumulate(map(ord, s2)))]
        for i, row in enumerate(s1[1: ], 1):
            dp.append([ord(row) + dp[-1][0]])
            for j, col in enumerate(s2[1: ], 1):
                if s1[i] == s2[j]:
                    dp[-1].append(dp[i - 1][j - 1])
                else:
                    dp[-1].append(min(dp[i - 1][j] + ord(s1[i]), dp[i][j - 1] + ord(s2[j])))
#         print(dp)
        return dp[-1][-1]

不要保存整个表，因为`list.append`比较慢。可以快那么一点点。

In [48]:
import itertools
class Solution:
    def minimumDeleteSum(self, s1, s2):
        """
        :type s1: str
        :type s2: str
        :rtype: int
        """
        s1 = "\x00" + s1
        s2 = "\x00" + s2
        dp = list(itertools.accumulate(map(ord, s2)))
        for i, row in enumerate(s1[1: ], 1):
            thisRow = [ord(row) + dp[0]]
            for j, col in enumerate(s2[1: ], 1):
                if s1[i] == s2[j]:
                    thisRow.append(dp[j - 1])
                else:
                    thisRow.append(min(dp[j] + ord(s1[i]), thisRow[j - 1] + ord(s2[j])))
            dp = thisRow
#         print(dp)
        return dp[-1]

In [49]:
Solution().minimumDeleteSum("delete", "leet")

403

In [50]:
Solution().minimumDeleteSum("a", "b")

195

## 646. Maximum Length of Pair Chain

这道题我没有找答案，自己想出来的（终于！）感觉做多了，对动态规划有点感觉了。一看到题目就有种强烈的[Maximum Subarray](#53.-Maximum-Subarray)感。

先以左边界排序（我也不知道能不能按右边界排序）。设$a_i, b_i$分别是第i个区间的左边界和右边界。设$p_i$是**必须以第i个区间为结尾的**最长的chain的数量。那么$p_i$和前面的项有什么联系呢？

发现，如果第i-1个区间的右边界小于第i个区间的左边界的话，chain就可以+1，比如
```
..., (1, 2), (3, 4)
```
这样$p_i = p_{i - 1} + 1$。

但是如果第i-1个区间的右边界大于等于第i个区间的左边界的话，chain就不能直接+1，因为和前面紧邻的区间没办法加起来，比如
```
..., (1, 2), (2, 3)
```
那这个时候怎么办？只能再往前看一格，看第i-2个区间的右边界满不满足右边界小于第i个区间的左边界
```
..., (0, 1), (1, 2), (2, 3)
```
很好！可以，那么$p_i = p_{i - 2} + 1$。

可是如果第i-2个区间的右边界还是不满足怎么办？再往前看一格。第i-3个区间的右边界还是不满足怎么办？再往前看一格...第1个区间的右边界还是不满足怎么办？不能再往前看一格了，那就只能$p_i = 1$了。

总结出递推式
$$
    p_i = \left\{\begin{aligned}
        p_{i - 1} + 1, &\qquad b_{i - 1} < a_i \\
        p_{i - 2} + 1, &\qquad b_{i - 2} < a_i \\
        \vdots \\
        p_1 + 1, &\qquad b_1 < a_i \\
        1, &\qquad \text{otherwise}
    \end{aligned}\right.
$$

初始条件很简单，就是$p_1 = 1$。

In [13]:
class Solution:
    def findLongestChain(self, pairs):
        """
        :type pairs: List[List[int]]
        :rtype: int
        """
        if pairs == []:
            return 0
        pairs = sorted(pairs, key=lambda x: x[0]) # 先排序
        dp = [1]
        for i, (left, right) in enumerate(pairs[1: ], 1):
            for j, _ in list(enumerate(pairs[: i]))[:: -1]: # 倒序查找
                if left > pairs[j][-1]: # 找到了
                    dp.append(dp[j] + 1)
                    break
            else: # 迭代一遍没找到
                dp.append(1) # 只能是1了
#         print(dp)
        return max(dp)

In [18]:
Solution().findLongestChain([[1,2], [2,3], [3,4]])

2

In [19]:
Solution().findLongestChain([[1,2], [2,3], [3,4], [4,5]])

2

还看到一种很简短的做法。但是为什么要按右边界排序、为什么能cover所有情况，我还不理解。

据说按右边界排序的原因是贪心法。

In [17]:
import operator
class Solution:
    def findLongestChain(self, pairs):
        """
        :type pairs: List[List[int]]
        :rtype: int
        """
        maximumRight, maximumChainLength = float('-inf'), 0 # cur是目前最大右边界，res是目前最大chain长度
        for p in sorted(pairs, key=operator.itemgetter(1)): # 按右边界排序
            if maximumRight < p[0]: 
                maximumRight, maximumChainLength = p[1], maximumChainLength + 1
        return maximumChainLength

## 64. Minimum Path Sum

教科书式dp。

设$p_{i, j}$是到达$(i, j)$格点的最小花费。那么$p_{i, j}$和前面的项有什么关系？

非常简单，因为要达到$(i, j)$格点，只能
- 要么从$(i - 1, j)$往下面走一格
- 要么从$(i, j - 1)$往右边走一格

所以递推式是
$$
    p_{i, j} = \min\{p_{i - 1, j}, p_{i, j - 1}\} + a_{i, j}
$$
初始条件
$$
\left\{\begin{aligned}
    p_{0, j} &= \sum_{k = 0}^j p_{0, k} \\
    p_{i, 0} &= \sum_{k = 0}^i p_{k, 0}
\end{aligned}\right.
$$

In [52]:
import itertools
class Solution:
    def minPathSum(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        lastRow = list(itertools.accumulate(grid[0]))
        for i, row in enumerate(grid[1: ], 1):
            thisRow = [lastRow[0] + row[0]]
            for j, col in enumerate(row[1: ], 1):
                thisRow.append(min(thisRow[-1], lastRow[j]) + col)
            lastRow = thisRow
        return lastRow[-1]

In [53]:
Solution().minPathSum([
  [1,3,1],
  [1,5,1],
  [4,2,1]
])

7

In [54]:
Solution().minPathSum([
  [1]
])

1

## 486. Predict the Winner

被[Stone Game](#877.-Stone-Game)坑过一次之后，我觉得这道题也可以转化成Stone Game来做。而且它说了平局算先手赢。

所以如果
- nums长度是偶数，这题就变成Stone Game了，所以先手必赢
- nums长度是奇数

  情况有点麻烦，有可能后手会赢。要先转化成偶数情况。
  
  首先要计算除去头或尾之后，剩下的元素的奇数偶数下标和。算出之后，后手一定会选大的那个，但是会出现两种情况
  - 后手在小局里赢了之后，数量比先手大，那么后手赢
  - 后手在小局里赢了之后，数量小于等于先手，那么先手赢
  
  先手在这种情况下，可以做的事情是
  - 假装自己选头，然后计算出剩下的奇数偶数和的差，发现
  
    - 差大于头，那么后手一定选大的那个，自己必输
    - 差小于等于头，那么后手即使选了大的那个，也抵不上自己选的，所以还是自己赢
      
  - 假装自己选尾，然后计算出剩下的奇数偶数和的差，发现
  
    - 差大于尾，那么后手一定选大的那个，自己必输
    - 差小于等于尾，那么后手即便选了大的那个，也抵不上自己选的，所以还是自己赢
    
可是这题的Test case有一点奇怪。在`[0,0,7,6,5,6,1]`的时候，先手应该是必赢的。

In [61]:
class Solution:
    def PredictTheWinner(self, nums):
        """
        :type nums: List[int]
        :rtype: bool
        """
        if len(nums) % 2 == 0:
            return True
        else:
            if (abs(sum(nums[1: ][:: 2]) - sum(nums[2: ][:: 2])) > nums[0]) \
            and (abs(sum(nums[: -1][1: ][:: 2]) - sum(nums[: -1][:: 2])) > nums[-1]):
                return False
            return True

In [62]:
Solution().PredictTheWinner([1, 5, 2])

False

In [63]:
Solution().PredictTheWinner([1, 5, 233, 7])

True

In [65]:
Solution().PredictTheWinner([0,0,7,6,5,6,1])

True

## 312. Burst Balloons

（膨胀了，开始做hard题了嘻嘻）

In [ ]:
class Solution:
    def maxCoins(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        

## 62. Unique Paths

https://leetcode.com/problems/unique-paths/description/

我感觉这道题是一道数学题（但是我没想出通项公式怎么求）。给你一个m x n的矩阵，有个机器人在左上角，终点在右下角，机器人只能往右或往下走，问你有几种走法。

设$p_{i, j}$是到达$(i, j)$的走法个数。要走到$(i, j)$，只能
- 要么从$(i - 1, j)$往下走一格
- 要么从$(i, j - 1)$往右走一格

所以递推式显然是
$$
    p_{i, j} = p_{i - 1, j} + p_{i, j - 1}
$$

边界条件是
$$
\left\{\begin{aligned}
    p_{0, 0} &= 0 \\
    p_{0, j} &= 1 \\
    p_{i, 0} &= 1
\end{aligned}\right.
$$

DP做法。

In [71]:
class Solution:
    def uniquePaths(self, m, n):
        """
        :type m: int
        :type n: int
        :rtype: int
        """
        if n == 1 or m == 1:
            return 1
        lastRow = [0] + [1] * (n - 1)
        for i in range(1, m):
            thisRow = [1]
            for j in range(1, n):
                thisRow.append(thisRow[-1] + lastRow[j])
            lastRow = thisRow
        return lastRow[-1]

In [73]:
Solution().uniquePaths(3, 3)

6

我觉得是有通项公式的……

## 63. Unique Paths II

加了障碍物，无非就是多加个判断。

In [130]:
class Solution:
    def uniquePathsWithObstacles(self, obstacleGrid):
        """
        :type obstacleGrid: List[List[int]]
        :rtype: int
        """
        if len(obstacleGrid) == 1: # one row
            if 1 in obstacleGrid[0]:
                return 0
            else:
                return 1
        if len(obstacleGrid[0]) == 1: # one column
            if any(1 in i for i in obstacleGrid):
                return 0
            else:
                return 1
            
        if obstacleGrid[0][0] == 1:
            return 0
            
        lastRow = [0]
        for j in range(1, len(obstacleGrid[0])):
            if j == 1:
                if obstacleGrid[0][1] == 1:
                    lastRow.append(0)
                else:
                    lastRow.append(1)
            else:
                if obstacleGrid[0][j] == 1:
                    lastRow.append(0)
                else:
                    lastRow.append(lastRow[-1])
                
#         print(lastRow)
                
        for i in range(1, len(obstacleGrid)):
            if i == 1:
                if obstacleGrid[1][0] == 1:
                    thisRow = [0]
                else:
                    thisRow = [1]
            else:
                if obstacleGrid[i][0] == 1:
                    thisRow = [0]
                else:
                    thisRow = [lastRow[0]]
                
            for j in range(1, len(obstacleGrid[0])):
                if obstacleGrid[i][j] == 1:
                    thisRow.append(0)
                else:
                    thisRow.append(thisRow[-1] + lastRow[j])
            lastRow = thisRow
#             print(lastRow)
        return lastRow[-1]

In [131]:
Solution().uniquePathsWithObstacles([
  [0,0,0],
  [0,1,0],
  [0,0,0]
])

2

In [132]:
Solution().uniquePathsWithObstacles([
  [1,0],
  [0,0],
])

0

In [133]:
Solution().uniquePathsWithObstacles([[0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],[0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0],[1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,1],[0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0],[0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0],[1,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0],[0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0],[0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0],[0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1],[0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0],[0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0],[0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1],[1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,1],[0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,1],[1,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1],[0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0]])

1637984640

但是上面的代码写的实在是太难看了……居然也打败了99%。

## 740. Delete and Earn

这道题和[House Robber](#198.-House-Robber)有点像，都是不能取相邻的东西。

问题在于，这道题需要先做个直方图，再按横轴从小到大排序，再用dp。

设$a_i$是直方图的横轴按从小到大排序之后，第i个数。比如`[2, 2, 3, 3, 4]`这个数列，做完直方图、再按横轴排序之后是
```
item #
----------
2    2
3    3
4    1
```
意思是数列里出现了三个数2, 3, 4。它们就是直方图的横轴。那么排完序后就是`2, 3, 4`。那么$a_0$就是2，$a_1$是3，$a_2$是4。

设$p_i$是算上直方图里第i个数之后，当前为止最大的收益。那么$p_i$和前面的项有什么关系？
- 首先如果$a_i$和前面一个数$a_{i - 1}$并不相邻，那根本不用考虑什么拿不拿的问题，直接拿就完事儿了，反正拿了也不可能冲突，那肯定是拿了之后收益更大
- 但是如果$a_i$和前面一个数$a_{i - 1}$相邻，那就很不幸了。要考虑拿不拿的问题
  
  - 如果拿$a_i$，那么前面一个数$a_{i - 1}$就不能拿，只能拿再前面一个$a_{i - 2}$，那么$p_i = p_{i - 2} + a_i \cdot \#{a_i}$
  - 如果不拿$a_i$，那么$p_i = p_{i - 1}$，无事发生
  
  所以最后要比一个大小，使$p_i$符合定义**到当前元素之后为止的最大收益**。
  
这样递推式就出来了
$$
    p_i = \left\{\begin{aligned}
        p_{i - 1} + a_i \cdot \# a_i, &\qquad a_i - a_{i - 1} > 1 \\
        \max\{p_{i - 2} + a_i \cdot \# a_i, p_{i - 1}\}, &\qquad a_i - a_{i - 1} = 1 \\
    \end{aligned}\right.
$$

接下来看初始条件。因为递推式里出现了$p_{i - 2}$，所以初始条件需要考虑$p_0, p_1$。

$$
\left\{\begin{aligned}
    p_0 &= a_0 \cdot \# a_i \\
    p_1 &= \left\{\begin{aligned}
        p_0 + a_1 \cdot \# a_1, &\qquad a_1 - a_0 > 1 \\
        \max\{p_0, a_1 \cdot \# a_1\}, &\qquad a_1 - a_0 = 1\\
    \end{aligned}\right.
\end{aligned}\right.
$$
或者你也可以在最前面填充一个0，让直方图从0开始。这样可以少写一个初始条件。

In [25]:
import collections
class Solution:
    def deleteAndEarn(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        if nums == []:
            return 0
        counter = collections.Counter(nums)
        if len(counter) == 1:
            return nums[0] * len(nums)
        
        indexes = sorted(counter.keys())
        dp = [indexes[0] * counter[indexes[0]]]
        if indexes[1] - indexes[0] == 1:
            dp.append(max(dp[0], indexes[1] * counter[indexes[1]]))
        else:
            dp.append(dp[0] + indexes[1] * counter[indexes[1]])
            
        for i in range(2, len(indexes)):
            if indexes[i] - indexes[i - 1] > 1: # 当前这个数和前面一个数并不相邻，所以直接加上就完事儿了
                dp.append(dp[-1] + indexes[i] * counter[indexes[i]])
            else: # 当前这个数和前面一个数相邻，残念
                dp.append(max(dp[-2] + indexes[i] * counter[indexes[i]], \
                             dp[-1]))
#         print(counter)
#         print(dp)
        return dp[-1]

In [34]:
Solution().deleteAndEarn([2, 2, 3, 3, 3, 4])

9

In [33]:
Solution().deleteAndEarn([2, 3, 4])

6

In [32]:
Solution().deleteAndEarn([1,6,3,3,8,4,8,10,1,3])

43

因为python的list的append操作非常慢，所以最好能不用list就不用list。这里我们发现递推式只与前面两项有关系，所以只要保存上一次和上上一次的结果就可以了。

这样能快很多。下面的代码直接打败了99.54 %。

In [31]:
import collections
class Solution:
    def deleteAndEarn(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        if nums == []:
            return 0
        counter = collections.Counter(nums)
        if len(counter) == 1:
            return nums[0] * len(nums)
        
        indexes = sorted(counter.keys())
        dp1 = indexes[0] * counter[indexes[0]]
        if indexes[1] - indexes[0] == 1:
            dp2 = max(dp1, indexes[1] * counter[indexes[1]])
        else:
            dp2 = dp1 + indexes[1] * counter[indexes[1]]
            
        for i in range(2, len(indexes)):
            if indexes[i] - indexes[i - 1] > 1: # 当前这个数和前面一个数并不相邻，所以直接加上就完事儿了
                temp = dp2 + indexes[i] * counter[indexes[i]]
            else: # 当前这个数和前面一个数相邻，残念
                temp = max(dp1 + indexes[i] * counter[indexes[i]], \
                             dp2)
            dp1 = dp2
            dp2 = temp
#         print(counter)
#         print(dp)
        return dp2

## 873. Length of Longest Fibonacci Subsequence

如果是substring还挺好做的……但是这里是subsequence，意思是取出来的子序列里面的每一个元素在原序列里面不一定要相邻。

[substring和subsequence的区别](https://en.wikipedia.org/wiki/Subsequence)

In [93]:
class Solution:
    def lenLongestFibSubseq(self, A):
        """
        :type A: List[int]
        :rtype: int
        """
        if len(A) < 3:
            return 1
        dp = [0, 0]
        for i, value in enumerate(A[2: ], 2):
            if value == A[i - 1] + A[i - 2]:
                dp.append(dp[-1] + 1)
            else:
                dp.append(1)
        return max(dp)

In [94]:
Solution().lenLongestFibSubseq([1,2,3,4,5,6,7,8])

1

-----

## 动态规划类总结

观察上面的几个题目，会发现它们基本上都是这样的形式：找到
$$
    \max\{f(i, j) | 0 \leq i \leq j \leq n - 1\}
$$

- 当$f(i, j) = \sum_{k = i}^{j} a_k$时，变成了Maximum Subarray问题;
- 当$f(i, j) = \min\{a_i, a_j\} \cdot (j - i)$，变成了Container with Most Water问题；
- 当$f(i, j) = a_j - a_i$，变成了Best Time to Buy and Sell Stock问题;
- 当$f(i, j) = \prod_{k = i}^{j} a_k$时，变成了Maximum Product Subarray问题；

### 何时答案是$\max\{P_i\}$、何时答案是$P_{n - 1}$？

### 该怎么选择合适的$P_i$的定义？

## 15. 3Sum

<https://leetcode.com/problems/3sum/description/>

本质上是要找到
$$
    \{(a_i, a_j, a_k) | 0 \leq i < j < k \leq n - 1, a_i + a_j + a_k = 0\}
$$

## 201. Bitwise AND of Numbers Range

https://leetcode.com/problems/bitwise-and-of-numbers-range/description/

比较浅显的办法是直接reduce。

In [362]:
import functools
class Solution(object):
    def rangeBitwiseAnd(self, m, n):
        """
        :type m: int
        :type n: int
        :rtype: int
        """
        return functools.reduce(lambda x, y: x & y, range(m, n + 1))

In [368]:
Solution().rangeBitwiseAnd(8, 17)

0

但是在数非常大的时候非常非常慢。

有一个规律。所有是2的次方的数都是最开头有一个1，然后接下来全都是0。所以如果一个区间里面
- 存在$2^n$

  这时有两种情况
  - 最大的$2^n$不是左边界
    
    意味着存在比$2^n$小的数，这些数的最前面是0，这样什么都不用看了，结果直接是0
    
  - 最大的$2^n$是左边界
  
    意味着没有比$2^n$小的数，所以没有最前面的0。这样结果直接就是这个$2^n$
    
- 不存在$2^n$
  
  好像只能reduce了。

[这里](https://www.cnblogs.com/grandyang/p/4431646.html)给出了一种非常巧妙的办法。通过找规律可以发现这个题目是找区间内所有的数的二进制表示中、从左边算起、最长的公共部分，找到最长公共部分之后，再在右边补完0。

方法是先建一个全1的mask。这里也要注意是$2^{31} - 1$而不是$2^{31}$。然后每次和上下边界bitwise and，如果发现两个结果相等了，~~说明这个mask就是了~~这是不对的，应该取这个相等的结果；如果不是，那么mask继续左移1位。如果mask左移到变成0了，就说明确实没有公共部分。

In [398]:
import functools
class Solution(object):
    def rangeBitwiseAnd(self, m, n):
        """
        :type m: int
        :type n: int
        :rtype: int
        """
        mask = 2 ** 31 - 1
        for offset in range(32):
            q = mask << offset
#             print(bin(q))
            if (q & m) == (q & n):
                return q & m
        return 0

In [399]:
Solution().rangeBitwiseAnd(0, 1)

0

最快的是这种。但是也很难理解。

又出现了`i & (i - 1)`这个幽灵。

In [ ]:
class Solution(object):
    def rangeBitwiseAnd(self, m, n):
        """
        :type m: int
        :type n: int
        :rtype: int
        """
        res = n
        while res > m:
            res = res & (res - 1)
        return res

[这个解释](https://leetcode.com/problems/bitwise-and-of-numbers-range/discuss/56719/JavaPython-easy-solution-with-explanation)也很不错。

In [ ]:
def rangeBitwiseAnd(self, m, n):
    i = 0
    while m != n:
        m >>= 1
        n >>= 1
        i += 1
    return n << i

## 461. Hamming Distance

https://leetcode.com/problems/hamming-distance/description/

hamming重量的意思是一个数的二进制表示中、1的个数。

可以逐位统计1的个数。先看这个数是不是单数，如果是，那么最低位必定是1；如果不是，最低位是0。统计完后，把数右移一位。直到这个数变成0。

In [142]:
class Solution(object):
    def hammingWeight(self, n):
        """
        :type n: int
        :rtype: int
        """
        ones = 0
        while n != 0:
            if n % 2:
                ones += 1
            n = n >> 1
        return ones

In [147]:
Solution().hammingWeight(3)

2

## 202. Happy Number

<https://leetcode.com/problems/happy-number/description/>

如果是快乐数，那么一定会出现1。如果不是快乐数，也没有必要一直while循环下去，而是会出现周期。所以需要开一个集合，如果见到1，直接返回True；如果不是1，先看下集合里有没有这个数、也就是看一下以前有没有出现过这个数字，如果出现过了，那就说明开始周期循环了，以后也没有必要再循环下去了，直接返回False。

In [152]:
class Solution(object):
    def isHappy(self, n):
        """
        :type n: int
        :rtype: bool
        """
        numbers = set()
        while True:
            n = sum(int(i) ** 2 for i in str(n))
            if n == 1:
                return True
            
            if n in numbers:
                return False
            else:
                numbers.add(n)

In [153]:
Solution().isHappy(19)

True

## 190. Reverse Bits

<https://leetcode.com/problems/reverse-bits/description/>

Python内建`bin`做法。

In [136]:
class Solution:
    # @param n, an integer
    # @return an integer
    def reverseBits(self, n):
        return int("{:0>32}".format(bin(n)[2: ])[:: -1], 2)

In [137]:
Solution().reverseBits(43261596)

964176192

另外建一个数，每次左移再加上n的最低位。左移32次。

有点像颠倒stack的方法。先建另一个stack，然后不停地pop原来的stack，pop出来的元素放到新的stack上。

In [148]:
class Solution:
    # @param n, an integer
    # @return an integer
    def reverseBits(self, n):
        q = 0
        for i in range(32):
            q = (q << 1) + (n & 1)
            n = n >> 1
        return q

In [149]:
Solution().reverseBits(43261596)

964176192

Leetcode显示第一种做法更快。

## 477. Total Hamming Distance

https://leetcode.com/problems/total-hamming-distance/description/

最naive的做法是生成两两对，然后计算每对的hamming距离，再加起来。

但是这肯定不是最快做法。所以我们找规律。发现就是统计在每一位上、有多少数在这一位上是0、有多少数在这一位上是1、然后这两个数字相乘、再把每一位的结果加起来。

In [76]:
class Solution(object):
    def totalHammingDistance(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        if nums == []:
            return 0
        summation = 0
        length = len(nums)
        for offset in range(max(nums).bit_length()):
            q = len([i for i in nums if (i >> offset) % 2 == 0])
            summation += q * (length - q)
        return summation

In [77]:
Solution().totalHammingDistance([4, 14, 2])

6

In [78]:
Solution().totalHammingDistance([])

0

下面这个的核心思想和我的一样，但是有一个小技巧，利用了`zip()`使“对位”更方便了。

In [82]:
class Solution(object):
    def totalHammingDistance(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        ans = 0
        n = len(nums)
        for i in zip(*map(lambda x: bin(x)[2: ].zfill(32), nums)):
            zero = i.count("0")
            ans += zero * (n-zero)
        return ans

In [83]:
Solution().totalHammingDistance([4, 14, 2])

6

## 470. Implement Rand10() Using Rand7()

https://leetcode.com/problems/implement-rand10-using-rand7/description/

本质上是一个随机变量$X$满足
$$
    P(X = k) = {1 \over 7}, \qquad \forall k \in \{1, 2, 3, 4, 5, 6, 7\}
$$

两次`rand7`调用可以看做是完全独立的，所以第二次调用可以看成是另一个随机变量$Y$，它和$X$独立同分布(i.i.d)。

这样我们的样本空间就可以扩展到49个状态，且它们的概率完全相同
$$
    P(X = k, Y = l) = {1 \over 49}, \qquad \forall \left(\begin{matrix} k & l \end{matrix}\right)^T \in \{1, 2, 3, 4, 5, 6, 7\}^2
$$

而我们仅仅要求有10个概率完全相同的状态。反正不怕状态少，怕的是状态不够多，所以最简单的方法是随便选定10个状态，如果某次测试，发现测试结果不在这10个状态里，就直接扔掉，下一次重新来。

In [195]:
import random
def rand7():
    return random.randint(1, 7)

In [210]:
# The rand7() API is already defined for you.
# def rand7():
# @return a random integer in the range 1 to 7

class Solution:
    def rand10(self):
        """
        :rtype: int
        """
        space = {(1, j): j for j in range(1, 8)}
        space.update({(2, 1): 8, (2, 2): 9, (2, 3): 10})
        while True:
            a = rand7()
            b = rand7()
            if not (a, b) in space:
                continue
            else:
                return space[(a, b)]

In [209]:
Solution().rand10()

9

## 172. Factorial Trailing Zeroes

https://leetcode.com/problems/factorial-trailing-zeroes/description/

数$n!$的末尾0的个数。直接算也是可以的但是太慢。

本质上是找到$n!$的因子里最多有多少2、 5对，因为一对2和5乘起来能在最后产生一个0。

但是观察发现2的因子数目永远大于等于5的因子数目，所以只要数小的那个、也就是5的因子个数就可以了。

In [254]:
class Solution:
    def trailingZeroes(self, n):
        """
        :type n: int
        :rtype: int
        """
        count = 0
        while n:
            q = n // 5
            count += q
            n = q
        return count

In [257]:
Solution().trailingZeroes(25)

6

递归做法。但是Python限制最大递归深度，所以不推荐。

In [ ]:
class Solution:
    def trailingZeroes(self, n):
        """
        :type n: int
        :rtype: int
        """
        if n == 0:
            return 0
        return n // 5 + self.trailingZeroes(n // 5)

## 343. Integer Break

https://leetcode.com/problems/integer-break/description/

题目要求，把一个整数$N$拆分成几个整数$a_1, a_2, \ldots, a_n$，使得$\prod_{k = 1}^n a_k$最大。

改写问题。已知
$$
    \sum_{k = 1}^n a_k = N
$$
那么
$$
    \prod_{k = 1}^n a_k \leq \left({N \over n}\right)^n
$$
等号在$a_1 = a_2 = \ldots = a_n = {N \over n}$时成立。

先研究下积的极大值。这里无耻的用一下导数（按理说是不能用的，因为$n$是整数）
$$
    {d \over dn} \left({N \over n}\right)^n = \left({N \over n}\right)^n (\ln N - \ln n - 1)
$$
极大值处$n$的取值是
$$
    n = {N \over e}
$$
极大值是
$$
    e^{N \over e}
$$
而且在极大值处，每份的大小是
$$
    e
$$

也就是说，尽量要把这个数切成每份接近$e$。最接近$e$的整数是3，所以要尽量把数字拆成每份都是3。

当然不是每个整数都能被3整除
- 被3除余1，这时候最后剩下的一个数是1，但是1对积没有帮助，所以不如再借一个3，和1拼成一个4
- 被3除余2，没问题

然后要当心n = 1, 2, 3的情况，这些是特殊情况。
- n = 1, 2的时候，最大是1
- n = 3的时候，最大是2

从4开始就能用上面的规则了。

In [46]:
class Solution:
    def integerBreak(self, n):
        """
        :type n: int
        :rtype: int
        """
        if n == 1 or n == 2:
            return 1
        if n == 3:
            return 2
        if n % 3 == 0:
            return round(3 ** (n / 3))
        elif n % 3 == 1:
            return 4 * 3 ** (n // 3 - 1)
        else:
            return 3 ** (n // 3) * 2

In [50]:
Solution().integerBreak(4)

4

## 421. Maximum XOR of Two Numbers in an Array

https://leetcode.com/problems/maximum-xor-of-two-numbers-in-an-array/description/

本质上要找到
$$
    \max\{a_i \oplus a_j | a_i, a_j \in A\}
$$

https://www.cnblogs.com/grandyang/p/5991530.html 的解法

In [6]:
class Solution:
    def findMaximumXOR(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        q = 0
        mask = 0
        for i in range(0, 32)[:: -1]:
            mask |= (1 << i)
            s = set()
            for num in nums:
                s.add(num & mask)
            t = q | (1 << i)
            for prefix in s:
                if t ^ prefix in s:
                    q = t
                    break
        return q

In [7]:
Solution().findMaximumXOR([10,23,20,18,28])

30

## 932. Beautiful Array

设数列$\{a_k\}$是$1, 2, 3, \ldots, N$。

$\{a'_k\}$是$\{a_k\}$的一种重排，并且
$$
    \forall 1 \leq i < k < j \leq N, \qquad 2 a_k \neq a_i + a_j
$$
试找出任意一种满足上面定义的重排。

In [1]:
class Solution:
    def beautifulArray(self, N):
        """
        :type N: int
        :rtype: List[int]
        """
        return sorted(range(1, N + 1), key=lambda x: bin(x)[2: ][:: -1])

上面我们已经从理论上找到了这样一种重排。根据一个数的二进制表示的倒序、按lexical顺序来排序，就能满足要求。

In [2]:
Solution().beautifulArray(4)

[4, 2, 1, 3]

In [3]:
Solution().beautifulArray(5)

[4, 2, 1, 5, 3]

## 877. Stone Game

这题绝了。

首先有一个博弈论定理叫<a href="https://en.wikipedia.org/wiki/Zermelo%27s_theorem_(game_theory)">Zermelo's theorem</a>。说在规则公平的双人回合制游戏里，先手必赢。

可能这个比较难理解。以这道题为例，首先石头有偶数堆、石头的总个数是奇数，所以不会存在平局。为什么？
- 石头有偶数堆，所以游戏结束时，两人手里的石头的堆数一定相同
- 石头的总个数是奇数，因为奇数只能由一个奇数加一个偶数，所以游戏结束时，一定是

  - 先手有偶数个石头
  - 后手有奇数个石头
  
  或者
  
  - 先手有奇数个石头
  - 后手有偶数个石头
  
  因为一个数不可能既是偶数又是奇数，所以先手和后手一定有人赢有人输，不会出现平局。

Alex是先手，那么他其实可以二选一
- 全部选index是偶数的石头堆0, 2, 4, ..., n - 2
- 全部选index是奇数的石头堆1, 3, 5, ..., n - 1

关键在于看index是偶数的石头堆的石头总数和index是奇数的石头堆的石头总数哪个大。哪个大，Alex就选哪个。假如index是偶数的石头堆加起来石头总数大于index是奇数的石头堆加起来石头总数，那么Alex就全选偶数。

因为上面说过了，石头总数是奇数，所以只可能出现二选一的情况
- index是偶数的石头堆加起来石头总数是偶数，index是奇数的石头堆加起来石头总数是奇数
- index是偶数的石头堆加起来石头总数是奇数，index是奇数的石头堆加起来石头总是是偶数

既然一个数不能又是偶数又是奇数，所以必然会出现二选一的情况
- index是偶数的石头堆加起来石头总数大
- index是奇数的石头堆加起来石头总数大

所以Alex算一下index是偶数的石头堆加起来的石头总数、index是奇数的石头堆加起来的石头总数，看哪个大，就一直选偶数或奇数就能稳赢。

可是，Alex能全部选index是偶数的石头堆、或者全部选index是奇数的石头堆吗？Bob会允许他这样做吗？Bob有阻止Alex的办法吗？

不幸的是，Bob还真没有。因为Alex有先手优势，所以他能为所欲为，而且Bob根本阻止不了他。

我们假设游戏开始前，Alex经过计算，发现index是偶数的石头堆加起来石头总数是比较大的，于是他决定全选偶数堆。这样他第一回合就会选第0堆石头收入囊中。此时，场上的石头堆只有第1到第n-1堆，Bob只能选择第1堆或者第n-1堆，而1和n-1都是奇数！那么如果
- Bob选了第1堆。于是下一回合Alex可以选择第2堆或第n-1堆。而Alex一定会选择第2堆
- Bob选了第n-1堆。于是下一回合Alex可以选择第1堆或第n-2堆。而Alex一定会选择第n-2堆

Bob想要逆转局势根本不可能。

In [ ]:
class Solution:
    def stoneGame(self, piles):
        """
        :type piles: List[int]
        :rtype: bool
        """
        return True

## 494. Target Sum

In [ ]:
class Solution:
    def findTargetSumWays(self, nums, S):
        """
        :type nums: List[int]
        :type S: int
        :rtype: int
        """
        

## 数学类总结

## 306. Additive Number

In [ ]:
class Solution:
    def threeSum(self, nums):
        """
        :type nums: List[int]
        :rtype: List[List[int]]
        """
        if len(nums) < 3:
            return False
        start = 0
        n1 = 1
        n2 = 2
        n3 = 3

## 回溯类总结

## 17. Letter Combinations of a Phone Number

<https://leetcode.com/problems/letter-combinations-of-a-phone-number/description/>

In [1]:
import itertools
class Solution:
    def letterCombinations(self, digits):
        """
        :type digits: str
        :rtype: List[str]
        """
        dictionary = {
            "2": list("abc"),
            "3": list("def"),
            "4": list("ghi"),
            "5": list("jkl"),
            "6": list("mno"),
            "7": list("pqrs"),
            "8": list("tuv"),
            "9": list("wxyz")
        }
#         dictionary = dict(zip(list('23456789'),'abc def ghi jkl mno pqrs tuv wxyz'.split()))
#         这样写更好
        return ["".join(i) for i in itertools.product(*[dictionary[j] for j in digits])]

In [2]:
Solution().letterCombinations("23")

['ad', 'ae', 'af', 'bd', 'be', 'bf', 'cd', 'ce', 'cf']

## 20. Valid Parentheses

<https://leetcode.com/problems/valid-parentheses/description/>

In [16]:
class Solution:
    def isValid(self, s):
        """
        :type s: str
        :rtype: bool
        """
        stack = []
        pairs = {
            "(": ")",
            "[": "]",
            "{": "}"
        }
        for i in s:
            if i in ["(", "[", "{"]:
                stack.append(i)
            else:
                if stack == []:
                    return False
                elif i != pairs[stack[-1]]:
                    return False
                else:
                    stack.pop()
        if stack == []:
            return True
        else:
            return False

In [20]:
Solution().isValid("]")

False

## 12. Integer to Roman

<https://leetcode.com/problems/integer-to-roman/description/>

In [3]:
class Solution:
    def intToRoman(self, num):
        """
        :type num: int
        :rtype: str
        """
        M = ["", "M", "MM", "MMM"] #千
        C = ["", "C", "CC", "CCC", "CD", "D", "DC", "DCC", "DCCC", "CM"] #百
        X = ["", "X", "XX", "XXX", "XL", "L", "LX", "LXX", "LXXX", "XC"] #十
        I = ["", "I", "II", "III", "IV", "V", "VI", "VII", "VIII", "IX"] #个
        return M[num//1000] + C[(num%1000)//100] + X[(num%100)//10] + I[num%10]

In [4]:
Solution().intToRoman(1994)

'MCMXCIV'

## 13. Roman to Integer

<https://leetcode.com/problems/roman-to-integer/description/>

## 171. Excel Sheet Column Number

<https://leetcode.com/problems/excel-sheet-column-number/description/>

In [33]:
class Solution:
    def titleToNumber(self, s):
        """
        :type s: str
        :rtype: int
        """
        return sum((ord(value) - 64) * 26 ** index for index, value in enumerate(s[:: -1]))

In [39]:
Solution().titleToNumber("ZZ")

702

## 168. Excel Sheet Column Title

<https://leetcode.com/problems/excel-sheet-column-title/description/>

171题反过来。设数字是$m$，那么$m$可以表示成
$$
    m = a_{n - 1} \cdot 26^{n - 1} + a_{n - 1} \cdot 26^{n - 2} + \ldots + a_0 \cdot 26^0
$$
其中权重数$a_i$可以这么算出来
$$
\begin{align}
    a_{n - 1} &= m \mod 26^{n - 1} \\
    a_{n - 2} &= (m - a_{n - 1} \cdot 26^{n - 1}) \mod 26^{n - 2} \\
    a_{n - 3} &= (m - a_{n - 1} \cdot 26^{n - 1} - a_{n - 2} \cdot 26^{n - 2}) \mod 26^{n - 3} \\
    \vdots \\
    a_k &= \left(m - \sum_{i = k + 1}^{n - 1} a_k \cdot 26^i\right) \mod 26^k \\
    \vdots \\
    a_0 &= \left(m - \sum_{i = 1}^{n - 1} a_i \cdot 26^i\right) \mod 26^0
\end{align}
$$
但是excel这个计数法则是不正确的，用上面的方法是**错误的，**因为没有0。你想，1是A，26是Z，但是27是AA，而不是A0。A又当0又当1。

```cpp
class Solution {
public:
    string convertToTitle(int n) {
        string ret = "";
        while(n)
        {
            ret = (char)((n-1)%26+'A') + ret;
            n = (n-1)/26;
        }
        return ret;
    }
};
```

In [133]:
class Solution:
    def convertToTitle(self, n):
        """
        :type n: int
        :rtype: str
        """
        s = ""
        while n:
            s = chr((n - 1) % 26 + ord("A")) + s
            n = (n - 1) // 26
        return s

In [134]:
Solution().convertToTitle(702)

'ZZ'

## 205. Isomorphic Strings

<https://leetcode.com/problems/isomorphic-strings/description/>

验证A和B之间是否存在一个一一对应的映射。

In [165]:
class Solution(object):
    def isIsomorphic(self, s, t):
        """
        :type s: str
        :type t: str
        :rtype: bool
        """
        return list(map(s.find, s)) == list(map(t.find, t))

In [166]:
Solution().isIsomorphic("papee", "title")

False

上面的解法本质上是把字符串序列替换成其中每个字符在这个字符串里第一次出现的位置组成的序列，然后比较两个字符串变换后的序列是否相同。这个解法真的很巧妙，我非常喜欢。

为什么验证每个字符第一次出现的位置是否相同和问题是等价的呢?我也没法严格证明这个问题。

In [167]:
class Solution(object):
    def isIsomorphic(self, s, t):
        """
        :type s: str
        :type t: str
        :rtype: bool
        """
        return len(set(s)) == len(set(t)) == len(set((i, j) for i, j in zip(s, t)))

In [169]:
Solution().isIsomorphic("paper", "title")

True

上面的解法是我想出来的直观的解法。速度稍微慢一点。

前一个等号`len(set(s)) == len(set(t))`保证了两个序列里不会有出现元素种类数目不同。也就是不会出现`abc, abb`这种情况；第二个等号保证了即使元素种类数目相同、同一个也元素不会映射两次，比如`aba, baa`这种情况。

## 219. Contains Duplicate II

<https://leetcode.com/problems/contains-duplicate-ii/description/>

In [5]:
class Solution(object):
    def containsNearbyDuplicate(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: bool
        """
#         if k + 1 > len(nums):
#             k = len(nums) - 1
#         for i in range(len(nums) - k):
#             print(nums[i: i + k + 1])
#             if k + 1 != len(set(nums[i: i + k + 1])):
#                 return True
#         return False
        dictionary = {}
        for index, value in enumerate(nums):
            if value in dictionary: # 如果值已经在里面的话，就判断j - i <= k。
                if index - dictionary[value] <= k:
                    return True # 说明找到了
            dictionary[value] = index # 不管在不在，都需要更新最近坐标
        return False # 找了一圈都没有

In [6]:
Solution().containsNearbyDuplicate([1,2,3,1], 3)

True

In [7]:
Solution().containsNearbyDuplicate([1,0,1,1], 1)

True

In [8]:
Solution().containsNearbyDuplicate([1,2,3,1,2,3], 2)

False

In [9]:
Solution().containsNearbyDuplicate([99, 99], 2)

True

## 263. Ugly Number

<https://leetcode.com/problems/ugly-number/description/>

总之先把所有的数都生成出来存在set里面，再判断就非常非常快了。

In [21]:
import itertools
numbers = {2**i * 3**j * 5**k for i, j, k in itertools.product(range(32), range(21), range(15))}
numbers = {i for i in numbers if i <= 2**31}

class Solution(object):
    def isUgly(self, num):
        """
        :type num: int
        :rtype: bool
        """
        return num in numbers

In [24]:
Solution().isUgly(64)

True

还有一种很巧妙的办法，用数论的方法。因为
$$
    n = 2^i \cdot 3^j \cdot 5^k, \qquad i \in [1, 31], j \in [1, 20], k \in [1, 14]
$$
所以$n$一定能被$2^{31} \cdot 3^{20} \cdot k^{14}$整除，因为
$$
    {2^{31} \cdot 3^{20} \cdot k^{14} \over 2^i \cdot 3^j \cdot 5^k} = 2^{31 - i} \cdot 3^{20 - j} \cdot 5^{14 - k} \in \mathbb{Z}
$$

In [27]:
class Solution(object):
    def isUgly(self, num):
        """
        :type num: int
        :rtype: bool
        """
        #n = (2**31)*(3**20)*(5**14) = 4570198050078720000000000000
        return False if num < 1 or (45701980500787200000000000000) % num != 0 else True

In [33]:
Solution().isUgly(16)

True

这个速度非常快。数学对算法真的是降维打击。

## 342. Power of Four

<https://leetcode.com/problems/power-of-four/description/>

In [57]:
class Solution(object):
    def isPowerOfFour(self, num):
        """
        :type num: int
        :rtype: bool
        """
        return True if 4294967296 % num == 0 else False

In [65]:
Solution().isPowerOfFour(-2147483648)

True

## 290. Word Pattern

<https://leetcode.com/problems/word-pattern/description/>

做这个和上一题[一一对应映射的题](#205.-Isomorphic-Strings)有似曾相识的感觉。

实际上解法一模一样。

In [34]:
class Solution(object):
    def wordPattern(self, pattern, str):
        """
        :type pattern: str
        :type str: str
        :rtype: bool
        """
        str = str.split(" ")
        return list(map(pattern.index, pattern)) == list(map(str.index, str))

In [36]:
Solution().wordPattern("aaaa", "dog cat cat dog")

False

In [54]:
class NumArray(object):

    def __init__(self, nums):
        """
        :type nums: List[int]
        """
        if nums == []:
            self.empty = True
            return
        else:
            self.empty = False
        self.nums = nums
        self.integral = [nums[0]]
        for i in nums[1: ]:
            self.integral.append(self.integral[-1] + i)

    def sumRange(self, i, j):
        """
        :type i: int
        :type j: int
        :rtype: int
        """
        if self.empty == True:
            return 0
        if i == 0:
            return self.integral[j]
        return self.integral[j] - self.integral[i - 1]


# Your NumArray object will be instantiated and called as such:
# obj = NumArray(nums)
# param_1 = obj.sumRange(i,j)

In [55]:
NumArray([-2, 0, 3, -5, 2, -1]).sumRange(2, 5)

-1

In [56]:
NumArray([]).sumRange(2, 5)

0

## 367. Valid Perfect Square

<https://leetcode.com/problems/valid-perfect-square/description/>

In [85]:
class Solution(object):
    def isPerfectSquare(self, num):
        """
        :type num: int
        :rtype: bool
        """
        if num == 0:
            return True
        n = 1
        while n ** 2 <= num:
            if num == n ** 2:
                return True
            n += 1
        return False # 超过了，不再算了

有点看不懂下面这种解法。

In [ ]:
class Solution(object):
    def isPerfectSquare(self, num):
        """
        :type num: int
        :rtype: bool
        """
        start, end = 0, num
        while start < end - 1:
            mid = (start + end)/2
            if mid*mid == num:
                return True
            if mid*mid > num:
                end = mid
            else:
                start = mid
        if start*start == num or end*end == num:
            return True
        return False

In [89]:
# The guess API is already defined for you.
# @param num, your guess
# @return -1 if my number is lower, 1 if my number is higher, otherwise return 0
# def guess(num):

class Solution(object):
    def guessNumber(self, n):
        """
        :type n: int
        :rtype: int
        """
        left = 0
        right = n + 1
        while True:
            currentGuess = int(round((left + right) / 2))
            result = guess(currentGuess)
            if result == 0:
                break
            elif result == 1:
                left = currentGuess
            else:
                right = currentGuess
                
        return currentGuess

In [98]:
Solution().guessNumber(2)

2

## 400. Nth Digit

<https://leetcode.com/problems/nth-digit/description/>

这玩意叫[Champernowne constant](https://en.wikipedia.org/wiki/Champernowne_constant)。

相关资料
- http://oeis.org/A033307
- http://oeis.org/A007376

但是貌似没有通项公式（有就好了直接秒杀）。那就只能来慢慢想了。

最笨的办法就是把这个数列直接弄出来。然后直接以n取下标就好了。可是上限是$2^31$，如果每个字符只占一个字节，那么也要大概2 GB空间才能存下这个数列。

那我们就看看能不能来优化。我们至少要先判断出**第n个字符所在的那个数字的位数。**这里有个规律。
- `1, 2, 3, ..., 9`一共9个1位数。n从1到9
- `10, 11, 12, ..., 99`一共90个2位数。n从10到189
- `100, 101, 102, ..., 999`一共900个3位数。n从189到...（算不出了）
- ...
- `1000...0, 1000...1, 1000...2, ..., 999...9`一共$9 \cdot 10^{i - 1}$个$i$位数。n从$\sum_{k = 1}^i 9 \cdot 10^{k - 1} \cdot k$到$\sum_{k = 1}^{i + 1} 9 \cdot 10^{k - 1} \cdot k$

这样就非常好办了。一下子就能判断出第n个字符所在的那个数字到底是一个几位数。

但是判断出来几位数还没完，还要再细化到**第n个字符所在的那个数字到底是几。**这一步不如上一步难，因为我们已经判断出第n个字符所在的那个数字是几位数了，而这个数字所在的区间里的所有数字的位数都是相同的。

In [54]:
class Solution:
    def findNthDigit(self, n):
        """
        :type n: int
        :rtype: int
        """
        digitsThisInterval = 1
        accumulativeDigitsBefore = 0
        while True:
            accumulativeDigitsUntilThisInterval = accumulativeDigitsBefore + 9 * 10 ** (digitsThisInterval - 1) * digitsThisInterval
            if n <= accumulativeDigitsUntilThisInterval:
                break
            else:
                accumulativeDigitsBefore = accumulativeDigitsUntilThisInterval
                digitsThisInterval += 1
        
        firstNumberThisInterval = 10 ** (digitsThisInterval - 1)
        thisNumber = (n - accumulativeDigitsBefore - 1) // digitsThisInterval + firstNumberThisInterval
#         print(thisNumber)
        return int(str(thisNumber)[n - accumulativeDigitsBefore - digitsThisInterval * (thisNumber - firstNumberThisInterval) - 1])

In [58]:
Solution().findNthDigit(190)

100


1

## 447. Number of Boomerangs

<https://leetcode.com/problems/number-of-boomerangs/description/>

本质上就是找到
$$
    \left\{(a_i, a_j, a_k) \vphantom{\sum}\middle| |a_i - a_j| = |a_i - a_k| \right\}
$$
其中
$$
    a_i, a_j, a_k \in \{a_1, a_2, \ldots, a_n\}, a_i \neq a_j \neq a_k
$$

## 459. Repeated Substring Pattern

<https://leetcode.com/problems/repeated-substring-pattern/description/>

In [4]:
class Solution(object):
    def repeatedSubstringPattern(self, s):
        """
        :type s: str
        :rtype: bool
        """
        for i in range(1, int(len(s) / 2) + 1):
            if len(s) % i == 0:
                if s[0: i] * round(len(s) / i) == s:
                    return True
        return False

In [10]:
Solution().repeatedSubstringPattern("abcabcabca")

False

还有非常非常巧妙的办法。先把字符串$S$复读一遍，变成$S' = \{S, S\}$，再去掉第一个字符、去掉最后一个字符，变成$S''$。再判断$S$是不是$S''$的子字符串。

这个方法我是不可能想出来的，但是我可以给出证明，证明这个方法是能区分复读字符串和非复读字符串的。注意，世界上的字符串要么是复读字符串，要么不是复读字符串，不存在既不是复读字符串也不是非复读字符串的字符串，也不存在既是复读字符串又是非复读字符串的字符串。这一点很重要。所以这个方法的目的是找到一种分割。

充分条件：证明这个方法能识别出复读字符串。证明方法是假设原字符串就是复读字符串，我们需要证明出这个方法能够得出`s in (s + s)[1: -1]`的结论。

假如一个字符串$S$是由一段长度为$p$的字符串重复$n$次得到的（$n > 1$），那么把这个整个字符串再复制一遍，变成$S' = \{S, S\}$，就会是长度为$p$的字符串重复$2 n$次得到的。然后再去掉第一个和最后一个字符，变成$S''$,这样$S''$里面含有$p$重复$2n - 2$次形成的字符串。

因为$n > 1$，所以$2n - 2 > 0$，即$2n - 2 \geq 1$，意思是$S''$里一定至少会出现一次$S$。

所以上面的证明证明了对于复读字符串，这个方法能够识别出来。但是如果一个字符串明明是非复读字符串，这个方法会不会误判为复读字符串呢？接着看下面的必要性证明。

必要条件：证明这个方法还能识别出非复读字符串。证明方法是假设这个方法得出了`s in (s + s)[1: -1]`的结果，证明这个字符串是复读字符串。

假设原字符串$S = \{a_i\} = \{a_1, a_2, \ldots, a_n\}$。复读一遍之后变成了
$$
    S' = \{a_i, a_i\} = \{\underbrace{a_1, a_2, \ldots, a_n}_{S}, \underbrace{a_1, a_2, \ldots, a_n}_{S}\}
$$
再掐头去尾变成了
$$
    S'' = \{a_2, \ldots, a_n, a_1, a_2, \ldots, a_{n- 1}\}
$$
现在存在一个$2 \leq k \leq n$，使得$S''$的一个子字符串
$$
\begin{align}
    &\{a_k, a_{k + 1}, \ldots, a_n, a_1, \ldots, a_{k - 1}\} \\
    = &\{a_1, a_2, \ldots, a_n\}
\end{align}
$$
即（写的好看一点）
$$
\begin{align}
    a_k &= a_1 \\
    a_{k + 1} &= a_2 \\
    a_{k + 2} &= a_3 \\
    \vdots \\
    a_n &= a_{n - k + 1} \\
    a_1 &= a_{n - k + 2} \\
    a_2 &= a_{n - k + 3} \\
    \vdots \\
    a_n &= a_{k - 1}
\end{align}
$$

据说这个叫KMP算法。

In [13]:
class Solution:
    def repeatedSubstringPattern(self, s):
        """
        :type s: str
        :rtype: bool
        """
        return s in (s + s)[1: -1]

In [12]:
Solution().repeatedSubstringPattern("abcabc")

True

## 

In [18]:
class Solution:
    def thirdMax(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        nums = set(nums)
        return list(sorted(nums))[-3] if len(nums) >= 3 else max(nums)

In [21]:
Solution().thirdMax([1, 2, 3])

1

## 434. Number of Segments in a String

<https://leetcode.com/problems/number-of-segments-in-a-string/description/>

In [22]:
class Solution:
    def countSegments(self, s):
        """
        :type s: str
        :rtype: int
        """
        return len(s.split())

In [23]:
Solution().countSegments("Hello, my name is John")

5

## 338. Counting Bits

<https://leetcode.com/problems/counting-bits/description/>

查了下，这个数列叫hamming weight数列。

相关资料
- http://oeis.org/A000120

通项公式
$$
    a_n = n - \sum_{k = 1}^n \left\lfloor n \over 2^k\right\rfloor
$$

In [130]:
class Solution:
    def countBits(self, num):
        """
        :type num: int
        :rtype: List[int]
        """
        return [bin(i).count("1") for i in range(num + 1)]

In [131]:
Solution().countBits(5)

[0, 1, 1, 2, 1, 2]

不使用`bin()`的办法。这个有一点DP的感觉。

首先定义$i$二进制表示中1的个数为$p_i$。那么$p_i$和前面的项有什么关系呢？

这里用的技巧是`i & (i - 1)`。其实这个技巧更多时候用来判断一个数是不是2的指数。凡是2的指数的数都有一个特点，就是最前面有一个1，然后后面都是0。这个不容易想出方法来判断。但是我们知道凡是2的指数的数减去1之后，就变成了从某个位置开始往右全是1。
$$
\begin{align}
\begin{aligned}
    2^0 &= 1_2 \\
    2^1 &= 10_2 \\
    2^2 &= 100_2 \\
\end{aligned}\qquad
\begin{aligned}
    2^0 - 1 &= 0_2 \\
    2^1 - 1 &= 1_2 \\
    2^2 - 1 &= 11_2 \\
\end{aligned}
\end{align}
$$
所以凡是2的指数的数，`i & (i - 1)`必等于0。

而“不是2的指数的数，`i & (i - 1)`必不等于0”这个怎么证明？

回到正题，$i$的二进制表示中1的个数就是$i \& (i - 1)$的二进制表示中1的个数加1。即
$$
    p_i = p_{i \& (i - 1)} + 1
$$

为什么会这样？

In [139]:
class Solution:
    def countBits(self, num):
        """
        :type num: int
        :rtype: List[int]
        """
        if num == 0:
            return [0]
        if num == 1:
            return [0, 1]
        q = [0, 1]
        for i in list(range(num + 1))[2: ]:
            print(i & (i - 1))
            q.append(q[i & (i - 1)] + 1)
        return q

In [137]:
Solution().countBits(5)

0
2
0
4


[0, 1, 1, 2, 1, 2]

## 300. Longest Increasing Subsequence

<https://leetcode.com/problems/longest-increasing-subsequence/description/>

设包含了第i个元素的递增subsequence最大长度是$p_i$。$p_i$和前面的项有什么关系呢？

$$
p_i = \left\{\begin{aligned}
    1 + \max\{p_j | a_j < a_i, j < i\}, &\qquad \{p_j | a_j < a_i, j < i\} \neq \emptyset \\
    1, &\qquad \{p_j | a_j < a_i, j < i\} = \emptyset
\end{aligned}\right.
$$

初始条件很简单，
$$
p_0 = 1
$$

In [116]:
class Solution:
    def lengthOfLIS(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        if nums == []:
            return 0
        dp = [1]
        for i, value in enumerate(nums[1: ], 1):
            dp.append(max([dp[j] + 1 for j, _ in list(enumerate(nums[: i])) if _ < value], default=1))
#         print(dp)
        return max(dp)

In [114]:
Solution().lengthOfLIS([10,9,2,5,3,7,101,18])

4

In [115]:
Solution().lengthOfLIS([10])

1

In [111]:
Solution().lengthOfLIS([1,3,6,7,9,4,10,5,6])

[1, 2, 3, 4, 5, 3, 6, 4, 5]


6

## 357. Count Numbers with Unique Digits

https://leetcode.com/problems/count-numbers-with-unique-digits/description/

找规律
- $n = 1$时，$[0, 9)$总共10个
- $n = 2$时，$[0 - 9)$总共10个,$[10, 100)$总共$9 \times 9 = 81$个
- $n = 3$时，$[0 - 9)$总共10个,$[10, 100)$总共$9 \times 9 = 81$个，$[100, 1000)$总共$9 \times 9 \times 8 = 648$个

所以找到递推式
$$
\left\{\begin{align}
    p_i &= p_{i - 1} + \underbrace{9 \times 9 \times 8 \times 7 \times \ldots}_{i个} = p_{i - 1} + 9 \cdot {9! \over [9 - (i - 1)]!}\\
    p_0 &= 1 \\
\end{align}\right.
$$

In [165]:
import math
class Solution:
    def countNumbersWithUniqueDigits(self, n):
        """
        :type n: int
        :rtype: int
        """
        if n == 0:
            return 1
        p = [1]
        for i in range(1, n + 1):
            p.append(p[-1] + 9 * round(math.factorial(9) / math.factorial(9 - (i - 1))))
        return p[-1]

In [166]:
Solution().countNumbersWithUniqueDigits(3)

739

## 392. Is Subsequence

https://leetcode.com/problems/is-subsequence/description/

判断`s`是不是`t`的一个subsequence（不一定要连续）

那就一个一个来判断吧。首先判断`s[0]`在不在`t`里面，
- 如果不在

  那么肯定不是subsequence，直接False
  
- 如果在

   那么好，找到`s[0]`在`t`里面第一次出现的位置`k`，那么`s[0]`就搞定了，`t[0: k + 1]`也直接不用看了，因为即便`s[1]`出现在`t[0: k + 1]`里面也没有意义。所以我们把`s`截断成`s[1: ]`，`t`也截断成`t[k + 1: ]`。
   
   然后这个过程反复重复，如果最后`s`变成了空字符串，意味着`s`里面的每一个元素都在`t`里找到了，而且是按`s`里的顺序在`t`中出现的，也就符合了subsequence的定义。

为什么是找第一次出现的位置呢？我也很难说清楚严谨的道理，但是总感觉这是个自然的想法。

In [100]:
class Solution:
    def isSubsequence(self, s, t):
        """
        :type s: str
        :type t: str
        :rtype: bool
        """
        while s != "":
            if s[0] in t:
                t = t[t.index(s[0]) + 1: ]
                s = s[1: ]
            else:
                return False
        return True

居然打败了100%。

这个做法没有用dp的思想（吧）

In [101]:
Solution().isSubsequence("ace", "abcde")

True

In [102]:
Solution().isSubsequence("b", "c")

False

In [104]:
Solution().isSubsequence("axc", "ahbgdc")

False

## 46. Permutations

https://leetcode.com/problems/permutations/description/

In [182]:
import itertools
class Solution:
    def permute(self, nums):
        """
        :type nums: List[int]
        :rtype: List[List[int]]
        """
        return [list(i) for i in itertools.permutations(nums)]

In [183]:
Solution().permute([1, 2, 3])

[[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3, 1, 2], [3, 2, 1]]

## 47. Permutations II

https://leetcode.com/problems/permutations-ii/description/

这题出现了重复的数，但是结果中却又不能出现重复的数。

比如`[1, 1, 2]`，结果中`[1, 1, 2], [1, 1, 2]`算一个。

In [184]:
import itertools
class Solution:
    def permuteUnique(self, nums):
        """
        :type nums: List[int]
        :rtype: List[List[int]]
        """
        return [list(i) for i in set(itertools.permutations(nums))]

In [186]:
Solution().permuteUnique([1, 1, 2])

[[1, 2, 1], [2, 1, 1], [1, 1, 2]]

## 48. Rotate Image

https://leetcode.com/problems/rotate-image/description/

一个n阶方阵$M$顺时针旋转等价于先转置，再相对垂直线镜面翻转。

In [214]:
class Solution:
    def rotate(self, matrix):
        """
        :type matrix: List[List[int]]
        :rtype: void Do not return anything, modify matrix in-place instead.
        """
        n = len(matrix)
        for i in range(n):
            for j in range(i, n):
                matrix[i][j], matrix[j][i] = matrix[j][i], matrix[i][j]
                print(i, j)
        
        for i in range(n):
            for j in range(n // 2):
                matrix[i][j], matrix[i][n - j - 1] = matrix[i][n - j - 1], matrix[i][j]
                print(i, j)
                
        print(matrix)

In [215]:
Solution().rotate([
  [1,2,3],
  [4,5,6],
  [7,8,9]
])

0 0
0 1
0 2
1 1
1 2
2 2
0 0
1 0
2 0
[[7, 4, 1], [8, 5, 2], [9, 6, 3]]


## 27. Remove Element

https://leetcode.com/problems/remove-element/description/

In [221]:
class Solution:
    def removeElement(self, nums, val):
        """
        :type nums: List[int]
        :type val: int
        :rtype: int
        """
        pos = []
        for index, value in enumerate(nums):
            if value == val:
                pos.append(index)
#         print(pos)
        for i in pos[:: -1]:
            nums[i: i + 1] = []
        return len(nums)
        

In [222]:
Solution().removeElement([3, 2, 2, 3], 3)

[0, 3]


2

这种写法也很漂亮。

In [ ]:
class Solution:
    def removeElement(self, nums, val):
        """
        :type nums: List[int]
        :type val: int
        :rtype: int
        """
        while val in nums:
            nums.remove(val)
        return len(nums)

## Search Range

In [223]:
class Solution:
    def searchRange(self, nums, target):
        """
        :type nums: List[int]
        :type target: int
        :rtype: List[int]
        """
        q = [-1, -1]
        for index, value in enumerate(nums):
            if value == target:
                if q[0] == -1: # met for the first time
                    q = [index, index]
                else: # met once before
                    q[1] = index
        return q

In [225]:
Solution().searchRange([5,7,7,8,8,10], 6)

[-1, -1]

## 55. Jump Game

https://leetcode.com/problems/jump-game/description/

跳到最后一级、第$n - 1$级的方法有$n - 1$种
- 从第$n - 2$级跳1级
- 从第$n - 3$级跳2级
- 从第$n - 4$级跳3级
- ...
- 从第0级跳$n - 1$级

In [ ]:
class Solution:
    def canJump(self, nums):
        """
        :type nums: List[int]
        :rtype: bool
        """
        

## 54. Spiral Matrix

https://leetcode.com/problems/spiral-matrix/description/

这边我用递归的思想来做的。对于任意一个矩阵
$$
\left[\begin{matrix}
    a_{0, 0} & a_{0, 1} & \ldots & a_{0, n - 1} \\
    \vdots & \ddots & & \vdots \\
    a_{m - 1, 0} & a_{m - 1, 1} & \ldots & a_{m - 1, n - 1}
\end{matrix}\right]
$$
首先输出第一行、从$a_{0, 0}$到$a_{0, n - 1}$，再输出最后一列、从$a_{1, n - 1}$到$a_{m - 1, n - 1}$。这样这个矩阵的上、右就处理完了，接下来应该处理下、左。

发现处理下、左的方式也可以用同一段代码处理，只不过这个矩阵要先处理一下：
1. 先去掉刚才处理的上、右，得到一个子矩阵
1. 显微镜对称一下、也就是先上下颠倒一下、再左右颠倒一下

这样再递归调用以下，把所有结果拼起来就好了。再加点极端case处理。

In [246]:
class Solution:
    def spiralOrder(self, matrix):
        """
        :type matrix: List[List[int]]
        :rtype: List[int]
        """
        if not matrix:
            return []
        rows = len(matrix)
        columns = len(matrix[0])
        if rows == 1:
            return matrix[0]
        if columns == 1:
            return [i[0] for i in matrix]
        subMatrix = [i[: -1] for i in matrix[1: ]]
#         print(subMatrix)
#         print([i[columns - 1] for i in matrix[1: ]])
#         print([i[:: -1] for i in subMatrix[:: -1]])
        return matrix[0] + [i[columns - 1] for i in matrix[1: ]] + self.spiralOrder([i[:: -1] for i in subMatrix[:: -1]])

In [243]:
Solution().spiralOrder([
  [1, 2, 3, 4],
  [5, 6, 7, 8],
  [9,10,11,12]
])

[1, 2, 3, 4, 8, 12, 11, 10, 9, 5, 6, 7]

In [244]:
Solution().spiralOrder([
 [ 1, 2, 3 ],
 [ 4, 5, 6 ],
 [ 7, 8, 9 ]
])

[1, 2, 3, 6, 9, 8, 7, 4, 5]

In [247]:
Solution().spiralOrder([])

[]

In [248]:
Solution().spiralOrder([[]])

[]

## 59. Spiral Matrix II

https://leetcode.com/problems/spiral-matrix-ii/description/

In [325]:
class Solution:
    def generateMatrix(self, n):
        """
        :type n: int
        :rtype: List[List[int]]
        """
        matrix = [[n ** 2]]
        for i in range(2 * (n - 1)):
            # first rotate the matrix
            rows = len(matrix)
            columns = len(matrix[0])
            matrix = [[matrix[r][c] for r in range(rows)[:: -1]] for c in range(columns)]
#             print(i, matrix)
            # insert new line
            matrix.insert(0, list(range(matrix[0][-1] - rows, matrix[0][-1])))
        return matrix

In [333]:
Solution().generateMatrix(5)

0 [[25]]
1 [[25, 24]]
2 [[25, 22], [24, 23]]
3 [[24, 25, 20], [23, 22, 21]]
4 [[23, 24, 17], [22, 25, 18], [21, 20, 19]]
5 [[21, 22, 23, 14], [20, 25, 24, 15], [19, 18, 17, 16]]
6 [[19, 20, 21, 10], [18, 25, 22, 11], [17, 24, 23, 12], [16, 15, 14, 13]]
7 [[16, 17, 18, 19, 6], [15, 24, 25, 20, 7], [14, 23, 22, 21, 8], [13, 12, 11, 10, 9]]


[[1, 2, 3, 4, 5],
 [16, 17, 18, 19, 6],
 [15, 24, 25, 20, 7],
 [14, 23, 22, 21, 8],
 [13, 12, 11, 10, 9]]

## 73. Set Matrix Zeroes

https://leetcode.com/problems/set-matrix-zeroes/description/

In [340]:
class Solution:
    def setZeroes(self, matrix):
        """
        :type matrix: List[List[int]]
        :rtype: void Do not return anything, modify matrix in-place instead.
        """
        if matrix == [[]]:
            return [[]]
        rowWithZeros = set()
        columnWithZeros = set()
        # scan matrix
        for i in range(len(matrix)):
            for j in range(len(matrix[0])):
                if matrix[i][j] == 0:
                    rowWithZeros.add(i)
                    columnWithZeros.add(j)
                    
        for i in rowWithZeros:
            matrix[i] = [0] * len(matrix[0])
        for i in columnWithZeros:
            for j in matrix:
                j[i] = 0
#         return matrix

In [341]:
Solution().setZeroes([
  [0,1,2,0],
  [3,4,5,2],
  [1,3,1,5]
])

[[0, 0, 0, 0], [0, 4, 5, 0], [0, 3, 1, 0]]

In [342]:
Solution().setZeroes([
  [1,1,1],
  [1,0,1],
  [1,1,1]
])

[[1, 0, 1], [0, 0, 0], [1, 0, 1]]

## 74. Search a 2D Matrix

https://leetcode.com/problems/search-a-2d-matrix/description/

In [348]:
class Solution:
    def searchMatrix(self, matrix, target):
        """
        :type matrix: List[List[int]]
        :type target: int
        :rtype: bool
        """
        for index, value in enumerate(matrix):
            if not value:
                return False
            if value[0] <= target <= value[-1]:
                if target in value:
                    return True
                else:
                    return False
        return False

In [344]:
Solution().searchMatrix([
  [1,   3,  5,  7],
  [10, 11, 16, 20],
  [23, 30, 34, 50]
], 13)

False

In [345]:
Solution().searchMatrix([
  [1,   3,  5,  7],
  [10, 11, 16, 20],
  [23, 30, 34, 50]
], 3)

True

In [349]:
Solution().searchMatrix([[]], 3)

False

## 56. Merge Intervals

https://leetcode.com/problems/merge-intervals/description/

In [ ]:
# Definition for an interval.
# class Interval:
#     def __init__(self, s=0, e=0):
#         self.start = s
#         self.end = e

class Solution:
    def merge(self, intervals):
        """
        :type intervals: List[Interval]
        :rtype: List[Interval]
        """
        if len(intervals) == 0:
            return []
        q = [intervals[0]]
        for index, value in enumerate(intervals[1: ], 1):
            for interval in q:
                

In [350]:
import itertools
class Solution:
    def combine(self, n, k):
        """
        :type n: int
        :type k: int
        :rtype: List[List[int]]
        """
        return [list(i) for i in itertools.combinations(range(1, n + 1), k)]

In [351]:
Solution().combine(4, 2)

[[1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [3, 4]]

## 78. Subsets

https://leetcode.com/problems/subsets/description/

In [360]:
import itertools
class Solution:
    def subsets(self, nums):
        """
        :type nums: List[int]
        :rtype: List[List[int]]
        """
        return [list(i) for i in sum([list(itertools.combinations(nums, i)) for i in range(0, len(nums) + 1)], [])]

In [361]:
Solution().subsets([1, 2, 3])

[[], [1], [2], [3], [1, 2], [1, 3], [2, 3], [1, 2, 3]]

## Search Matrix II

In [426]:
class Solution(object):
    def searchMatrix(self, matrix, target):
        """
        :type matrix: List[List[int]]
        :type target: int
        :rtype: bool
        """
        if matrix == [[]] or matrix == []:
            return False
        if len(matrix[0]) == 1:
            for row in matrix:
                if row[0] == target:
                    return True
                if row[0] > target:
                    return False
            return False
        for row in matrix:
            if row[0] <= target <= row[-1]:
                if target in row:
                    return True
            if row[0] > target:
                return False
        return False

In [427]:
Solution().searchMatrix([
  [1,   4,  7, 11, 15],
  [2,   5,  8, 12, 19],
  [3,   6,  9, 16, 22],
  [10, 13, 14, 17, 24],
  [18, 21, 23, 26, 30]
], 0)

False

In [430]:
Solution().searchMatrix([[1], [2]], 0)

False

## 239. Sliding Window Maximum

https://leetcode.com/problems/sliding-window-maximum/description/

In [431]:
class Solution:
    def maxSlidingWindow(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: List[int]
        """
        

In [ ]:
Solution().maxSlidingWindow()

## 233. Number of Digit One

https://leetcode.com/problems/number-of-digit-one/description/

相关资料
- http://oeis.org/A094798

In [436]:
class Solution:
    def countDigitOne(self, n):
        """
        :type n: int
        :rtype: int
        """
        return sum(str(i).count("1") for i in range(n - 1))

In [437]:
Solution().countDigitOne(2**31)

KeyboardInterrupt: 

## 260. Single Number III

https://leetcode.com/problems/single-number-iii/description/

In [497]:
import collections
class Solution:
    def singleNumber(self, nums):
        """
        :type nums: List[int]
        :rtype: List[int]
        """
        counter = collections.Counter(nums)
        return [index for index, value in counter.items() if value == 1]

In [498]:
Solution().singleNumber([1, 2, 1, 3, 5, 2])

[3, 5]

## 287. Find the Duplicate Number

https://leetcode.com/problems/find-the-duplicate-number/description/

In [499]:
import collections
class Solution:
    def findDuplicate(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        counter = collections.Counter(nums)
        return [index for index, value in counter.items() if value != 1][0]

但是上面的解法空间复杂度不是$O(1)$。

## 274. H-Index

https://leetcode.com/problems/h-index/description/

In [512]:
class Solution:
    def hIndex(self, citations):
        """
        :type citations: List[int]
        :rtype: int
        """
        citations.sort(reverse=True)
        return max([min(citations[i], i + 1) for i in range(len(citations))], default=0)

In [513]:
Solution().hIndex([3,0,6,1,5])

3

## 342. Power of Four

https://leetcode.com/problems/power-of-four/description/

先看它是不是$2^k$，用上次的方法`i & (i - 1) == 0`判断。

然后再数二进制里面0的个数，如果是偶数个，那就是了。

In [515]:
class Solution:
    def isPowerOfFour(self, num):
        """
        :type num: int
        :rtype: bool
        """
        return True if num & (num - 1) == 0 and bin(num)[2: ].count("0") % 2 == 0 else False

In [518]:
Solution().isPowerOfFour(8)

False

还有一种很巧妙的办法是看这个数的底数为4的对数是不是整数。如果一个数$n$是$4^k$，那么
$$
    k = \log_4 n
$$

但是因为浮点数的精度问题，所以这个方法有局限。

In [519]:
import math
class Solution:
    def isPowerOfFour(self, num):
        """
        :type num: int
        :rtype: bool
        """
        return math.log(num, 4).is_integer()

In [532]:
Solution().isPowerOfFour(4**990)

False

上面这里就失效了。

## 326. Power of Three

https://leetcode.com/problems/power-of-three/description/

3为底有一点麻烦。

可以用`while`不停地除以三，如果最后变成了1，那就是了，如果中间突然不能整除3了，说明有杂质。

In [533]:
class Solution:
    def isPowerOfThree(self, n):
        """
        :type n: int
        :rtype: bool
        """
        if n <= 0:
            return False
        while True:
            if n == 1:
                return True
            if n % 3 != 0:
                return False
            else:
                n = n / 3

In [542]:
Solution().isPowerOfThree(28)

False

同样这个方法也可以用在前面的power of two, power of four问题里面。

还有非常巧妙的办法。

In [ ]:
class Solution:
    """
    T: O(1)
    S: O(1)

    - Use the maximum power of 3 to test current number
    - 3 ** 19 < 0x7FFFFFFF < 3 ** 20
    """
    def isPowerOfThree(self, n):
        """
        :type n: int
        :rtype: bool
        """
        return n > 0 and (3 ** 19) % n == 0

这也太赖皮了。

但是这个方法无法用在power of four里面。

## 372. Super Pow

https://leetcode.com/problems/super-pow/description/

In [ ]:
class Solution:
    def superPow(self, a, b):
        """
        :type a: int
        :type b: List[int]
        :rtype: int
        """
        

## 441. Arranging Coins

https://leetcode.com/problems/arranging-coins/description/

如果$n$是某个$k$的累加，那么
$$
    n = 1 + 2 + \ldots + k = {k (k + 1) \over 2}
$$
这时
$$
    k = {\sqrt{8 n + 1} \over 2} - {1 \over 2}
$$

所以答案是$\left\lfloor{\sqrt{8 n + 1} \over 2} - {1 \over 2}\right\rfloor$。

In [23]:
class Solution:
    def arrangeCoins(self, n):
        """
        :type n: int
        :rtype: int
        """
        if n == 0:
            return 0
        if n == 1:
            return 1
        k = 2
        while True:
            if (k * (k + 1)) >> 1 == n:
                return k
            if (k * (k + 1)) >> 1 > n:
                break
            else:
                k += 1
        return k - 1

In [30]:
Solution().arrangeCoins(7)

3

In [31]:
Solution().arrangeCoins(2089018456)

64637

但是上面这个算法很慢。

In [41]:
import math
class Solution:
    def arrangeCoins(self, n):
        """
        :type n: int
        :rtype: int
        """
        return int(math.sqrt(8 * n + 1) / 2 - 1 / 2)

In [45]:
Solution().arrangeCoins(2089018456)

64637

## 442. Find All Duplicates in an Array

https://leetcode.com/problems/find-all-duplicates-in-an-array/description/

In [46]:
import collections
class Solution:
    def findDuplicates(self, nums):
        """
        :type nums: List[int]
        :rtype: List[int]
        """
        counter = collections.Counter(nums)
        return [index for index, value in counter.items() if value == 2]

In [47]:
Solution().findDuplicates([4,3,2,7,8,2,3,1])

[3, 2]

## 451. Sort Characters By Frequency

https://leetcode.com/problems/sort-characters-by-frequency/description/

In [59]:
import collections
class Solution:
    def frequencySort(self, s):
        """
        :type s: str
        :rtype: str
        """
        counter = collections.Counter(s)
        return "".join(index * value for index, value in counter.most_common())

In [60]:
Solution().frequencySort("Aabb")

'bbAa'

## 463. Island Perimeter

https://leetcode.com/problems/island-perimeter/description/

我觉得不难，但是corner case特别特别多。test case也一定会测你函数的覆盖率。

In [ ]:
class Solution(object):
    def islandPerimeter(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        

## 456. 132 Pattern

https://leetcode.com/problems/132-pattern/description/

In [61]:
class Solution(object):
    def find132pattern(self, nums):
        """
        :type nums: List[int]
        :rtype: bool
        """
        if len(nums) < 3:
            return False
        maximums = [nums[0]]
        minimums = [nums[0]]
        for index, value in enumerate(nums[1: ], 1):
            if value > minimums[-1] and value < maximums[-1]:
                return True
            maximums.append(max(maximums[-1], value))
            minimums.append(min(minimums[-1], value))
        return False

In [62]:
Solution().find132pattern([1, 2, 3, 4])

False

In [63]:
Solution().find132pattern([3, 1, 4, 2])

True

In [64]:
Solution().find132pattern([-1, 3, 2, 0])

True

In [65]:
Solution().find132pattern([1,0,1,-4,-3])

True

## 468. Validate IP Address

https://leetcode.com/problems/validate-ip-address/description/

用正则表达式可以秒杀。

IPv4地址
```
(1?[1-9]?\d|2[0-4]\d|25[0-5])(\.(1?[1-9]?\d|2[0-4]\d|25[0-5])){3}$
```

IPv6地址
```
([0-9A-Fa-f]{1,4})(:([0-9A-Fa-f]{1,4})){7}$
```

In [107]:
import re
class Solution(object):
    def validIPAddress(self, IP):
        """
        :type IP: str
        :rtype: str
        """
        if bool(re.fullmatch(r"(1?[1-9]?\d|2[0-4]\d|25[0-5])(\.(1?[1-9]?\d|2[0-4]\d|25[0-5])){3}$", IP)):
            return "IPv4"
        elif bool(re.fullmatch(r"([0-9A-Fa-f]{1,4})(:([0-9A-Fa-f]{1,4})){7}$", IP)):
            return "IPv6"
        return "Neither"

In [108]:
Solution().validIPAddress("172.16.254.1")

'IPv4'

In [109]:
Solution().validIPAddress("255.255.255.255")

'IPv4'

In [110]:
Solution().validIPAddress("255.255.255.256")

'Neither'

In [111]:
Solution().validIPAddress("2001:0db8:85a3:0:0:8A2E:0370:7334")

'IPv6'

In [112]:
Solution().validIPAddress("01.01.01.01")

'Neither'

## 189. Rotate Array

https://leetcode.com/problems/rotate-array/description/

In [223]:
class Solution:
    def rotate(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: void Do not return anything, modify nums in-place instead.
        """
        length = len(nums)
        for i in range(k % length):
            nums.insert(0, nums.pop())
#         print(nums)

In [228]:
Solution().rotate([1,2,3,4,5,6,7], 1)

[7, 1, 2, 3, 4, 5, 6]


In [226]:
Solution().rotate([-1,-100,3,99], 2)

[3, 99, -1, -100]


切片方法。也是我一开始想到的方法。

In [242]:
class Solution:
    def rotate(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: void Do not return anything, modify nums in-place instead.
        """
        n = len(nums)
        k = k % n
        nums[:] = nums[n-k: ] + nums[: n-k]

In [243]:
Solution().rotate([1, 3], 1)

## 438. Find All Anagrams in a String

https://leetcode.com/problems/find-all-anagrams-in-a-string/description/

In [269]:
import collections
class Solution:
    def findAnagrams(self, s, p):
        """
        :type s: str
        :type p: str
        :rtype: List[int]
        """
        if len(p) > len(s):
            return []
        counterP = collections.Counter(p)
        lengthP = len(p)
        q = []
        for i in range(len(s) - lengthP + 1):
            counterS = collections.Counter(s[i: i + lengthP])
#             print(counterS)
            if counterS == counterP:
                q.append(i)
        return q

In [270]:
Solution().findAnagrams("cbaebabacd", "abc")

Counter({'c': 1, 'b': 1, 'a': 1})
Counter({'b': 1, 'a': 1, 'e': 1})
Counter({'a': 1, 'e': 1, 'b': 1})
Counter({'e': 1, 'b': 1, 'a': 1})
Counter({'b': 2, 'a': 1})
Counter({'a': 2, 'b': 1})
Counter({'b': 1, 'a': 1, 'c': 1})
Counter({'a': 1, 'c': 1, 'd': 1})


[0, 6]

In [271]:
Solution().findAnagrams("abab", "ab")

Counter({'a': 1, 'b': 1})
Counter({'b': 1, 'a': 1})
Counter({'a': 1, 'b': 1})


[0, 1, 2]

## 788. Rotated Digits

https://leetcode.com/problems/rotated-digits/description/

In [9]:
class Solution:
    def rotatedDigits(self, N):
        """
        :type N: int
        :rtype: int
        """
        return len([i for i in range(1, N + 1) if all(j not in str(i) for j in "018347")])

In [10]:
Solution().rotatedDigits(10)

4

In [12]:
class Solution:
    def rotatedDigits(self, N):
        """
        :type N: int
        :rtype: int
        """
        res = 0
        for n in range(1, N+1):
            if not all(c in "0125689" for c in str(n)):
                continue
            if any(c in "2569" for c in str(n)):
                res += 1
        return res

In [13]:
Solution().rotatedDigits(10)

4

## 796. Rotate String

https://leetcode.com/problems/rotate-string/description/

In [20]:
class Solution:
    def rotateString(self, A, B):
        """
        :type A: str
        :type B: str
        :rtype: bool
        """
        if A == "" and B == "":
            return True
        
        A = list(A)
        B = list(B)
        
        if len(A) != len(B):
             return False
            
        length = len(A)
        for i in range(length):
            if A == B:
                return True
            A.insert(0, A.pop())
        return False

In [21]:
Solution().rotateString("abcde", "bcdea")

True

In [22]:
Solution().rotateString("", "")

True

## 405. Convert a Number to Hexadecimal

https://leetcode.com/problems/convert-a-number-to-hexadecimal/description/

要求不能用内建的函数直接转换。

不知道用`struct`算不算符合要求。

In [39]:
import struct
class Solution:
    def toHex(self, num):
        """
        :type num: int
        :rtype: str
        """
        if num == 0:
            return "0"
        return struct.pack(">i", num).hex().lstrip("0")

In [33]:
Solution().toHex(-2)

'fffffffe'

In [34]:
Solution().toHex(0)

'0'

In [38]:
class Solution:
    def toHex(self, num):
        """
        :type num: int
        :rtype: str
        """
        if num == 0:
            return "0"
        ans = ""
        mp = "0123456789abcdef"
        for i in range(8):
            n = num & 15
            c = mp[n]
            ans = c + ans
            num = num >> 4
        return ans.lstrip("0")

## 647. Palindromic Substrings

https://leetcode.com/problems/palindromic-substrings/description/

In [45]:
class Solution:
    def countSubstrings(self, s):
        """
        :type s: str
        :rtype: int
        """
        return sum(1 if s[i: j] == s[i: j][:: -1] else 0 for i in range(0, len(s)) for j in range(i + 1, len(s) + 1))

In [46]:
Solution().countSubstrings("aaa")

6

上面做法是$O(n^3)$。应该还有更好的做法。

In [55]:
import collections
class Solution:
    def topKFrequent(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: List[int]
        """
        return [i[0] for i in collections.Counter(nums).most_common(k)]

In [57]:
Solution().topKFrequent([1, 1, 3, 3], 3)

[1, 3]

## 869. Reordered Power of 2

https://leetcode.com/problems/reordered-power-of-2/description/

In [77]:
import itertools
class Solution:
    def reorderedPowerOf2(self, N):
        """
        :type N: int
        :rtype: bool
        """
        for i in set(itertools.permutations(str(N))):
#             print(i)
            if i[0] == "0":
                continue
            q = int("".join(i))
            if q & (q - 1) == 0:
                return True
        return False

In [81]:
Solution().reorderedPowerOf2(850239671)

True

In [79]:
Solution().reorderedPowerOf2(121278734)

True

但是上面的做法会time limit exceed。

如果换个思路，我们不去检测每个组合，而是反过来，生成和N位数相同的2的指数，然后用`Counter`数每个数出现的次数。

发现$2^{31} > 10^9 $所以最多只要位移32次就好了。

In [96]:
import collections
class Solution:
    def reorderedPowerOf2(self, N):
        """
        :type N: int
        :rtype: bool
        """
        lengthN = len(str(N))
        counter = collections.Counter(str(N))
        targets = []
        for i in range(32):
            q = str(1 << i)
            if len(q) == lengthN:
                targets.append(q)
#         print(targets)
        if any(collections.Counter(i) == counter for i in targets):
            return True
        else:
            return False

In [97]:
Solution().reorderedPowerOf2(850239671)

['134217728', '268435456', '536870912']


True

In [98]:
Solution().reorderedPowerOf2(121278734)

['134217728', '268435456', '536870912']


True

In [99]:
Solution().reorderedPowerOf2(1)

['1', '2', '4', '8']


True

上面的方法非常好。

## 697. Degree of an Array

https://leetcode.com/problems/degree-of-an-array/description/

In [109]:
import collections
class Solution:
    def findShortestSubArray(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        counter = collections.Counter(nums)
        mostCommonTimes = counter.most_common(1)[0][1]
        mostCommons = [index for index, value in counter.most_common() if value == mostCommonTimes]
#         找到所有出现频率和最频繁出现的那个数字的出现频率相同的数字
#         print(mostCommons)
        return len(nums) - max(nums[:: -1].index(i) + nums.index(i) for i in mostCommons)

In [107]:
Solution().findShortestSubArray([1, 2, 2, 3, 1])

[1, 2]


2

In [108]:
Solution().findShortestSubArray([1, 2, 2, 3, 1, 4, 2])

[2]


6

## 554. Brick Wall

https://leetcode.com/problems/brick-wall/description/

搞一个`dict`，当然最好用`defaultdict`，记录距离墙最左端每个单位距离的缝隙的数量。遍历每行就好了。

还要注意最后处理一下corner case，也就是把最右的情况去掉。

In [123]:
import collections
class Solution:
    def leastBricks(self, wall):
        """
        :type wall: List[List[int]]
        :rtype: int
        """
        seeps = collections.defaultdict(int)
        width = sum(wall[0])
        for row in wall:
            for index, brick in enumerate(row):
                seeps[sum(row[0: index]) + brick] += 1
        del seeps[width]
#         print(seeps)
        return len(wall) - max(seeps.values(), default=0)

In [124]:
Solution().leastBricks([[1,2,2,1],
 [3,1,2],
 [1,3,2],
 [2,4],
 [3,1,2],
 [1,3,1,1]])

defaultdict(<class 'int'>, {1: 3, 3: 3, 5: 2, 4: 4, 2: 1})


2

In [126]:
Solution().leastBricks([
    [3],
    [3]
])

defaultdict(<class 'int'>, {})


2

一点小改动，可以快很多。

In [127]:
import collections
class Solution:
    def leastBricks(self, wall):
        """
        :type wall: List[List[int]]
        :rtype: int
        """
        seeps = collections.defaultdict(int)
        width = sum(wall[0])
        for row in wall:
            i = 0
            for index, brick in enumerate(row):
                i += brick
                seeps[i] += 1
        del seeps[width]
#         print(seeps)
        return len(wall) - max(seeps.values(), default=0)

In [128]:
Solution().leastBricks([
    [3],
    [3]
])

2

## 318. Maximum Product of Word Lengths

https://leetcode.com/problems/maximum-product-of-word-lengths/description/

要在`words`找到两个元素，使得它们没有共同字母、并且两个元素的长度的积最大。

暴力做法就是先筛选出所有没有共同字母的单词对，然后算他们的长度积，拿到最大的。复杂度是$O(n^2)$。

但是在筛选这个步骤可以做点优化。可以用一下`dict`，index是类似hash的东西，设置26位，每一位指示是否有a, b, c, ..., z这些字母，value存由同样字母组成的单词的最大长度。这样列举单词对的时候，对数也会少很多，因为很多单词的组成字母都是一样的。列举单词对的时候，取出的都是由同样字母组成的单词里最长的词的长度。

判断的时候更快更简单，直接按位与，如果两个单词组成字母没有重合的，按位与出来的结果是0。

In [189]:
import itertools
import collections
class Solution:
    def maxProduct(self, words):
        """
        :type words: List[str]
        :rtype: int
        """
        q = collections.defaultdict(int)
        for word in set(words):
            index = sum(1 << (ord(char) - 97) for char in set(word)) # `a`的ascii是97
            q[index] = max(q[index], len(word))
#         print(words)
        return max((a[1] * b[1] for a, b in itertools.combinations(q.items(), 2) if a[0] & b[0] == 0), default=0)

In [179]:
Solution().maxProduct(["abcw","baz","foo","bar","xtfn","abcdef"])

16

In [180]:
Solution().maxProduct([])

0

In [181]:
Solution().maxProduct(["a","aa","aaa","aaaa"])

0

In [182]:
Solution().maxProduct(["a","ab","abc","d","cd","bcd","abcd"])

4

In [183]:
Solution().maxProduct(["bdcecbcadca","caafd","bcadc","eaedfcd","fcdecf","dee","bfedd","ffafd","eceaffa","caabe","fbdb","acafbccaa","cdc","ecfdebaafde","cddbabf","adc","cccce","cbbe","beedf","fafbfdcb","ceecfabedbd","aadbedeaf","cffdcfde","fbbdfdccce","ccada","fb","fa","ec","dddafded","accdda","acaad","ba","dabe","cdfcaa","caadfedd","dcdcab","fadbabace","edfdb","dbaaffdfa","efdffceeeb","aefdf","fbadcfcc","dcaeddd","baeb","beddeed","fbfdffa","eecacbbd","fcde","fcdb","eac","aceffea","ebabfffdaab","eedbd","fdeed","aeb","fbb","ad","bcafdabfbdc","cfcdf","deadfed","acdadbdcdb","fcbdbeeb","cbeb","acbcafca","abbcbcbaef","aadcafddf","bd","edcebadec","cdcbabbdacc","adabaea","dcebf","ffacdaeaeeb","afedfcadbb","aecccdfbaff","dfcfda","febb","bfffcaa","dffdbcbeacf","cfa","eedeadfafd","fcaa","addbcad","eeaaa","af","fafc","bedbbbdfae","adfecadcabe","efffdaa","bafbcbcbe","fcafabcc","ec","dbddd","edfaeabecee","fcbedad","abcddfbc","afdafb","afe","cdad","abdffbc","dbdbebdbb"])

45

In [ ]:
class Solution(object):
    def maxProduct(self, words):
        d = {}
        for w in words:
            mask = 0
            for c in set(w):
                mask |= (1 << (ord(c) - 97))
            d[mask] = max(d.get(mask, 0), len(w))
        return max([d[x] * d[y] for x in d for y in d if not x & y] or [0])

暴力判断。time limit exceed。

In [184]:
import itertools
class Solution:
    def maxProduct(self, words):
        """
        :type words: List[str]
        :rtype: int
        """
        q = 0
        for word1, word2 in itertools.combinations(set(words), 2):
            if not set(word1).intersection(word2):
                q = max(q, len(word1) * len(word2))
        return q

In [185]:
Solution().maxProduct(["a","ab","abc","d","cd","bcd","abcd"])

4

In [186]:
Solution().maxProduct(["abcw","baz","foo","bar","xtfn","abcdef"])

16

## 599. Minimum Index Sum of Two Lists

https://leetcode.com/problems/minimum-index-sum-of-two-lists/description/

In [226]:
import collections
class Solution:
    def findRestaurant(self, list1, list2):
        """
        :type list1: List[str]
        :type list2: List[str]
        :rtype: List[str]
        """
        commons = set(list1).intersection(list2)
        ranking = collections.Counter({i: list1.index(i) + list2.index(i) for i in commons})
        minima = ranking.most_common()[:: -1][0][1]
        return [index for index, value in ranking.items() if value == minima]

In [227]:
Solution().findRestaurant(["Shogun", "Tapioca Express", "Burger King", "KFC"]
, ["Piatti", "The Grill at Torrey Pines", "Hungry Hunter Steakhouse", "Shogun"])

['Shogun']

In [228]:
Solution().findRestaurant(["Shogun", "Tapioca Express", "Burger King", "KFC"]
, ["KFC", "Shogun", "Burger King"])

['Shogun']

## 592. Fraction Addition and Subtraction

https://leetcode.com/problems/fraction-addition-and-subtraction/description/

In [307]:
import fractions
class Solution:
    def fractionAddition(self, expression):
        """
        :type expression: str
        :rtype: str
        """
        args = map(fractions.Fraction, expression.replace("+", " +").replace("-", " -").split())
        q = sum(args)
        return str(q.numerator) + "/" + str(q.denominator)

In [240]:
Solution().fractionAddition("-1/2+1/2")

'0/1'

In [308]:
Solution().fractionAddition("-1/6+1/7")

'-1/42'

## 846. Hand of Straights

https://leetcode.com/problems/hand-of-straights/description/

In [ ]:
class Solution:
    def isNStraightHand(self, hand, W):
        """
        :type hand: List[int]
        :type W: int
        :rtype: bool
        """
        if len(hand) % len(W) != 0:
            return False
        
        

## 507. Perfect Number

In [1]:
class Solution:
    def checkPerfectNumber(self, num):
        """
        :type num: int
        :rtype: bool
        """
        divisors = []
        for i in range(1, num // 2 + 1):
            if num % i == 0:
                divisors.append(i)
        return sum(divisors) == num

In [4]:
Solution().checkPerfectNumber(20996011)

False

上面的太慢了。

实际上perfect numbers是很少的很少的，少到人们怀疑是否有无限个perfect numbers。

https://en.wikipedia.org/wiki/Perfect_number

In [5]:
class Solution:
    def checkPerfectNumber(self, num):
        """
        :type num: int
        :rtype: bool
        """
        return num in {6, 28, 496, 8128, 33550336, 8589869056}

## 455. Assign Cookies

https://leetcode.com/problems/assign-cookies/description/

In [ ]:
class Solution:
    def findContentChildren(self, g, s):
        """
        :type g: List[int]
        :type s: List[int]
        :rtype: int
        """
        g = sorted(g)
        s = sorted(s)
        length = len(g)
        for cookie in s:
            if cookie >= 

## 684. Redundant Connection

https://leetcode.com/problems/redundant-connection/description/

## 498. Diagonal Traverse

https://leetcode.com/problems/diagonal-traverse/description/

In [10]:
class Solution:
    def findDiagonalOrder(self, matrix):
        """
        :type matrix: List[List[int]]
        :rtype: List[int]
        """
        indices = sorted(((i, j) for i in range(len(matrix)) for j in range(len(matrix[0]))), key=lambda x: x[0] + x[1] - x[(x[0] + x[1]) % 2] * 0.00000001)
        return [matrix[i][j] for i, j in indices]

In [11]:
Solution().findDiagonalOrder([
 [ 1, 2, 3 ],
 [ 4, 5, 6 ],
 [ 7, 8, 9 ]
]
)

[1, 2, 4, 7, 5, 3, 6, 8, 9]

## 89. Gray Code

https://leetcode.com/problems/gray-code/description/

https://en.wikipedia.org/wiki/Gray_code 这里有最快的方法。

```c
unsigned int BinaryToGray(unsigned int num)
{
    return num ^ (num >> 1);
}

/*
 * This function converts a reflected binary
 * Gray code number to a binary number.
 * Each Gray code bit is exclusive-ored with all
 * more significant bits.
 */
unsigned int GrayToBinary(unsigned int num)
{
    unsigned int mask = num >> 1;
    while (mask != 0)
    {
        num = num ^ mask;
        mask = mask >> 1;
    }
    return num;
}
```

In [12]:
class Solution:
    def grayCode(self, n):
        """
        :type n: int
        :rtype: List[int]
        """
        return [i ^ (i >> 1) for i in range(2 ** n)]

In [13]:
Solution().grayCode(2)

[0, 1, 3, 2]

In [14]:
Solution().grayCode(0)

[0]

## 717. 1-bit and 2-bit Characters

https://leetcode.com/problems/1-bit-and-2-bit-characters/description/

感觉可以用Moore状态机来做。

- `state0`表示检测到了0
  
  - 如果遇到1，进入`state1`
  - 如果遇到0，待在原地
  
- `state1`表示已经接收到了一个1，还在等待下一个字符
  
  - 如果再遇到一个1，进入`state11`
  - 如果遇到一个0，进入`state10`
  
- `state11`表示检测到了11

  - 如果遇到1，进入`state1`
  - 如果遇到0，进入`state0`
  
- `state10`表示检测到了10
    
  - 如果遇到1，进入`state1`
  - 如果遇到0，进入`state0`

In [16]:
class Solution:
    def isOneBitCharacter(self, bits):
        """
        :type bits: List[int]
        :rtype: bool
        """
        state0 = 0
        state1 = 1
        state11 = 2
        state10 = 3
        state = state0
        for i in bits:
            if state == state0:
                if i == 1:
                    state = state1
            elif state == state1:
                if i == 1:
                    state = state11
                elif i == 0:
                    state = state10
            elif state == state11:
                if i == 0:
                    state = state0
                elif i == 1:
                    state = state1
            elif state == state10:
                if i == 1:
                    state = state1
                elif i == 0:
                    state = state0
        return state == state0

In [17]:
Solution().isOneBitCharacter([1, 0, 0])

True

In [18]:
Solution().isOneBitCharacter([1, 1, 1, 0])

False

In [22]:
Solution().isOneBitCharacter([1, 0])

False

## 494. Target Sum

https://leetcode.com/problems/target-sum/description/

In [ ]:
class Solution:
    def findTargetSumWays(self, nums, S):
        """
        :type nums: List[int]
        :type S: int
        :rtype: int
        """
        

## 386. Lexicographical Numbers

https://leetcode.com/problems/lexicographical-numbers/description/

In [23]:
class Solution:
    def lexicalOrder(self, n):
        """
        :type n: int
        :rtype: List[int]
        """
        return sorted(range(1, n + 1), key=str)

## 857. Minimum Cost to Hire K Workers

https://leetcode.com/problems/minimum-cost-to-hire-k-workers/description/

In [ ]:
class Solution:
    def mincostToHireWorkers(self, quality, wage, K):
        """
        :type quality: List[int]
        :type wage: List[int]
        :type K: int
        :rtype: float
        """
        

##  820. Short Encoding of Words

In [1]:
class Solution:
    def minimumLengthEncoding(self, words):
        """
        :type words: List[str]
        :rtype: int
        """
        words = sorted(words, key=len)
        popIndexes = set()
        for index, value in enumerate(words[: -1]):
            for value2 in words[index + 1:]:
                if value2.endswith(value):
                    popIndexes.add(index)
        return sum([len(value) if index not in popIndexes else 0 for index, value in enumerate(words)]) + len(words) - len(popIndexes)

In [19]:
Solution().minimumLengthEncoding(["time", "me", "bell"])

10

In [20]:
Solution().minimumLengthEncoding(["time", "time", "time", "time"])

5

In [22]:
Solution().minimumLengthEncoding(["nydoptn","yayfxmu","aiprifz","yhgwgk","ahmisb","drwkqe","bdffcj","kyhxu","dibeapv","bijkq","pkdgi","fyufy","hbksgrv","pvgmoc","mmcebki","rymco","wdicegh","tdxwpws","fhgqaq","pvhnv","buvrttd","ssfjlzn","gobdr","gvzxvqu","qftzcyw","snlbc","wykqmb","lhqeod","hoakq","klynp","xktvp","mqvlw","txwtek","zyxyrmd","ziwclz","kyqnm","teaka","rwgob","ttfbim","hxiwyj","zlvstwe","rlqyiea","lcjlna","ficwcv","ncunyc","oeivcp","vbabub","wkjeakk","cccnn","zijhvk","eqofjf","dwejn","fgzmn","mdbko","fddep","objwjoj","nozppgq","qjvjzwo","skbtqt","hybuwq","vhojvxl","sbsoya","zuuxzn","vnhmh","ilquam","cwkax","eeafltm","vryqoy","fgggc","phnnsab","sfcqf","heyhn","thhdvng","uezij","ddkgw","tophaq","ygzzd","iajodf","hjkpb","kasguse","xlupcg","qnbzrl","uuuqeuv","uadhr","pqrxmdh","oykep","qeadmz","sudmjnw","tssxpd","toihkac","diyjpvs","remwdf","hmwbi","gffrjyv","ozhrex","xyzmilo","hddkpi","ctkgxnr","gfxyw","zzincl","hrynql","qzzqe","xvitfw","nieys","xpypq","rxkcs","qbiyb","qadcnq","fmebxns","zbdbtsa","uuinbkh","oefidb","zkxcv","ktwrnga","rflkuf","qsmhyw","bxvemr","uadxc","wiinrw","mzyfc","gihwxyp","vrbdjqr","mjddrvo","efbczf","ibjkr","dsohbz","ycauu","nqyxbl","bqroe","cnnzzy","owzcyi","dwcsgre","yfzhdx","qvatjf","kaoqbzn","kwupop","zkeidd","oouzy","nxckj","rxmyyz","sqntq","olkeub","bqjsm","eufael","ksixhfv","xuhanwe","cofwcbd","fzjcij","gpsjl","bapovu","obqre","uifqmsj","vbzzttp","tksmq","jtnrud","xflgah","efnctvf","esckym","nqjibdt","wnaezdp","uknidqs","lgwfgl","cpzbozd","dukhrq","zrjna","yjjrdx","vkilz","fmudp","pvtoz","yajjvm","iuakp","mnjzpd","hozwt","mtynp","rwnsei","mzjbbhg","ueyxbi","rwdzloh","aktlbgy","hrrpj","aejpjf","geuwwnd","zobuwr","tncsz","eegjet","qllgsl","rcyqtnx","ctgnwc","qfidyon","sskvc","hjxfqgl","zwtjx","rqcfgrp","hdhneaf","nkdgzfx","kslmfy","ekdpc","ttanaur","pktbwv","uwioi","spvcn","ysxhiab","gpbol","iftorp","jigcdbh","qitnldl","nakriw","qzdvgn","zsjcsh","uprgu","xkfjeyn","eqret","cpqxer","pwpwucl","jeuye","lcufq","kavqfk","nqwiroe","uiikdze","snxlwqv","fvuro","nleao","qwlnkog","twbzu","lmeerl","iluvy","wnekqb","vpuyt","sdxct","likghq","wrkbht","vyane","dbpjqol","vwylo","mcvrute","krgbbwz","tceomvv","agzzrf","pjvcji","xzxfxd","lmsywbm","zrgdjng","tajqw","pqdfld","vjwakmc","euvdk","ixmrzc","zewxii","kugorfj","rfyanwb","zdmcqpj","impgtsf","rwddxr","dykwre","xnhxgk","ieels","eyzbynm","uuklvh","yyxjcfy","nphqzvo","qguludy","fmstky","waqyunj","zzhgbne","adtsdl","adjnzlw","wuyhkia","fqnir","osppjl","tcwhx","fsare","cnkeo","kajfiq","bzzqp","bpffp","tkthlj","jtwfe","wuarz","oefhkeq","pwxhr","smkiupd","kwwxmrv","urkptza","wxeai","nkqos","nyxrb","aikjnl","efowc","kfkkzx","laqcga","zmcmda","ibwcx","xlcyfqp","luzctq","drmol","blcxn","zrvbfpl","upokwz","iddzazt","yghsgok","gqyaqw","flihmkt","padioya","ljcrmi","ddlqavg","liocmg","gspwjpq","rzdyut","sloknwi","hllgq","uiyszv","fviyzbc","gorwlho","stsilp","wdhjgg","yozyq","bckrpf","velto","hnxpo","umkyei","rvfsnp","yvkxhhp","ezaton","mqcvb","wqvgue","tusqbyt","wchzlt","ibbuvcm","opkjvuc","adgkwo","wusgua","mtqdj","epwhbe","vlfmbjd","luwwenu","ielwha","djrtxdt","ziyyh","btxkfe","hsxjsg","ajxyd","qphevu","hgpdou","kwgxb","idysvn","osswshn","cpxmqf","wdhsvou","kvqpwen","hezjd","xdfuipa","xzyhhee","agqnlog","aewbrzc","uaaam","wnxpto","xzwou","eyomid","ffalsz","djcsm","kxfanu","aqsseu","glrtwcn","eddiz","astsano","xsyii","ofasv","pbvue","isahv","cotyg","tcnhuio","yhvtgfe","tjxsu","lsgqbm","teddl","lwjlb","londd","ajilfg","arfdu","csjvp","bkkxure","xnzdao","uqlxp","bvwqejr","sqrjyrv","uzguf","rikez","hgwsvvt","srdyfpm","rbblpvw","qjywel","ygrhmz","gpblr","aubhdbv","kkefilk","abcos","ozvpumt","rrtft","ticit","dnssibi","rnvxc","mrfhiu","roywhsw","nlklh","ddery","ulqkj","ppvjge","gvlwfd","hyuxv","qxuxx","rnkov","zbuylcy","fclbi","uacho","pvlog","eebnace","ilwsmp","szkpe","qvyfq","aehbum","grrumf","iyrxoym","zqmab","zvoxjtf","wmgfz","yhlwmk","azthhn","xgeqe","bhrtr","uiwmhiy","imqzab","mxvqpq","wsogtd","pgmoehg","krmsgqw","zjgnk","mkvozhp","cfwhj","gktcvml","frjzdli","uepot","rnush","edygzgl","pzhyow","soqqv","dullbdf","ztbdo","dqkak","risvhf","tpegh","fdexde","sblkydw","obvvee","hdexqto","omuirn","msvzm","uwmsoj","ozfdne","behsi","yflmmnc","jcylxbp","woiba","dwvobn","sqmnqju","dikrvn","xqbrs","rywhaxp","gkypz","mmbqg","vcnnv","lcqrjcu","udenxrk","iouex","rxhla","oeufo","wfliy","lrfcpq","vkqjwr","ptufhfe","vboqdd","bdwtzef","zbmfboo","qscjw","gcmig","zcgoez","fzxtkot","zttpbc","ullojn","lgvdl","bupzsbf","xfiqe","qjfozyu","vimodg","yjuhns","exczoyp","kjuvhp","yacuuoj","kudtyp","hvfey","duvuj","wqlck","ltgwabr","tytfvjb","btbnr","ncbtd","ucbexch","qyurg","nlzmqlu","syyiuio","nynoa","jwbkzj","hmozpm","kfzqeu","tgkkty","pmhyoz","gjunk","heacc","awgos","fdtig","ylavbey","qazas","ccbrkb","vglswu","srkfjrl","igqqflt","vhfnl","whtwrj","dagotl","sjxmqqd","ceizi","wwcsjw","fglsj","ocydit","qcxkji","rpcuin","dyzurw","iuovjxc","cgojnru","bfkcn","mnjkti","vdskrvr","cigds","dasrzho","ukkgsq","hympgbt","uowyypx","dfcqpc","deavlou","ycngrex","wrsvry","lkzyczl","bwqczba","yrxpta","gnseu","oaifza","zgqhmt","cakcg","regdpy","btojyqt","mvymsl","xnutze","qqdgf","xgpldtl","zcdquaf","pdcbq","poxvbhm","krxewlg","plbbre","lifxikf","dwwenyl","ulmka","lxzpdbe","iecgc","cdaxdl","urjwun","hojyh","dmhfoe","ivbpbzu","svizs","msaqcag","fxrbdgg","rplyy","kjhobj","vwvdfv","bvoed","plqtvk","pbcmy","wcmxkyk","uzyzdu","wjtwpv","bnsskq","kyjmwfy","ayvafc","aekknl","kkemp","nummx","wcest","liaqg","mrkju","pkbjl","ooxkqxk","izssj","qnizr","bsxgu","ohwrjn","endyt","dqybmd","ckzpy","crqhb","eaqbg","uarexfa","hrlrbla","wrbume","fomlo","xhoght","dyivd","nowwvmb","lzgdipl","ptssel","clmuv","dispadv","ysdmkcu","gmtavy","biiuuj","muzcr","koyqm","inrentg","dbylwp","yfndruf","vxqnhy","setzr","ouxkb","nerwcng","vlyiv","sbqaers","xnjvbg","zragsva","vtjok","gdume","cocges","bqjzg","mcsut","hdvijr","jxkbeo","vsitiw","zhedhyt","dnxbk","kzrlj","yqqem","qdtel","cxcmouq","ypyiofo","zvodw","ovvfrq","xejsu","eercqku","osnshz","hosdif","gjqwvx","xkvkju","tzxto","ifwlqx","hixnjav","krgpc","garej","vkufnjc","ajmacqx","xaxso","frdvmem","kpjjwd","uocheqh","ediuhx","nwwyoi","jvyjaev","eplpce","jbhaal","iwdhthc","jnkusgt","caxvl","bvzqtyz","srpqqa","kzaogkv","fqdtga","jhcwhav","ffandne","nhxabcx","cyqmdcq","xaqcj","cicabs","foicwi","rhyfgvl","zqygsqh","woxfyhb","sfzfk","vweqqib","pmqlvkd","rvabt","aycsy","bvkvln","cqebt","tdsswc","lqrcpw","xuvoa","pdqzp","ropgr","cvajde","ttjixzz","ovwfzz","pxpum","grucrmv","kehxbz","sspwb","sfibzh","yrnvktb","epsdkit","iwylfs","jklfnt","llzlnjw","wtthn","whsog","fsnhmea","pibhfmo","ujvzdnr","wgcyc","kcrnfow","zjssihl","dsghve","mveuzx","duqdogr","rsayem","zpqzgh","dhpiai","vkapza","cqchscd","szergiw","povnuy","nckhv","yygbk","neptki","iqpjngb","eyrgs","utenun","vvyxzb","kanyau","psfuy","ezfxhi","auwcyz","tsrsug","uojiw","sionsjl","errwqse","yyxtn","uhaudph","ubpijk","cgmiwgs","mrwej","lcozwr","sczkn","mkinca","kajafa","sfspoqy","qabklg","yvduo","mnfqru","xvvdsca","oixtxzp","biayfd","tdwfhey","jsipdk","oxnxy","blzcn","detufsp","ybmnvs","dzisxn","bjjzb","fejeby","mcayl","eqkmm","kjnhmmg","dvthn","kqgyqy","bcfwy","ozhvx","jdkkfia","glctyce","thshyl","vqscn","mwfll","qiift","mpiqn","focwej","byxood","uooxir","authex","htaddtv","ezmuul","sgixtux","dzoqzg","cejof","tgnqg","ymlzj","tmxycig","htnimpg","guehuow","bgemiu","qkqcx","zdkzdls","zexmti","quyjoec","gfejoz","fxhsz","gbrtif","rukcnhy","ebrgbzf","vvnbxts","nkylpon","pobciji","juvpyim","rqkakuh","jbbbk","opdgq","lumjxea","hqezakn","kyfcm","btaec","rbysvzu","dxncqf","kvrpwri","zqsqipn","rjgga","obybdhk","rmgvlok","ghjiy","flwgvqw","ksawyhu","bvbav","eksugfh","svxhapr","zajlvi","chdxyr","oalkdm","jlcojsv","grynxo","crlnx","ojxcn","etuuo","pdygfbt","jaujv","qnguua","jltkjx","tntlra","obhpj","razxr","iejhfz","wotwr","pvhed","unqkz","atvcj","sfrnn","tpmwx","jqnsp","jnymzo","staiv","rlayct","tsdbstu","ayogdd","soegov","cjsodir","uzbvsn","qursx","vxtzez","fuqtt","qbesfut","mrwgj","edfky","uuuaeg","ipwwg","nqabvi","psnpz","wbqsqb","osnxwu","genfxx","njqvip","innrdgu","gehergm","zllqfij","jedgk","strdca","xlidkg","peqvz","qcgnvti","qnmsfzl","mjwhjbt","uqnsc","ncfxbm","tdotxqk","reqmr","ofvdx","kkzlp","gjvec","lmhguaj","oikix","cuegfel","eloutd","extvvzp","skwxuz","wqaeac","fskyavp","dcfyu","bsoijo","kfqdphk","wytlkc","fvrdtvv","hdjbifb","rsdhdn","nvqoiie","uznunom","kcfjads","pnqrdew","kljpxnk","jynulq","iudafgu","ljgunu","pqqtvtc","nyxmebq","zjmyrz","jmxhgta","knzhaeg","bsapi","fycgud","pzptqpf","hqkilzv","jyxssv","guzvct","auldf","nfarwhj","vmupo","utigdhp","origpr","zakhnuv","nnijrhz","whodoct","jrujrb","unwok","ymmlw","cpdmzen","bmgrjnu","ajlkanl","milnz","sdzky","vmjrkld","oybtfur","jgrfdi","dmjcbww","ktwomq","aajsczt","nfnccff","bwlvjuq","vdwjjrh","yiwpjw","dsrhzjn","tgfdv","uuzwqy","geryayy","yjqdag","gqksoiu","vrupk","slsah","belkdo","ntoxx","iwmupw","qatiwok","ahftce","ttpzi","cexqocr","klxzl","tnycg","qnfikw","hhpwc","vjjnyk","ybcocdi","ggcdfo","tiaow","jynar","oiueg","mnkvj","wcaxa","plhoqvq","xxzck","vzgiod","fqzabtq","ecegyq","bfcjcis","eeakg","qkdqt","amtnpj","ngzgj","cqnztaq","lvpmtup","idped","tfqmstr","deyktd","piirssq","jqcyn","txudy","dhjxng","demfkp","lvswa","tkfiu","yfgiwme","bqhfme","nahxru","yttdvwy","azypnj","ruflgr","drutc","ubvccx","kssau","zusprk","ifnpt","yqzymv","fwxsz","tyzbp","nskskw","sklygmn","sqgnv","gfmoo","oxqvper","wwxig","kxfhgb","vblps","uxqqto","limcbo","gkbtbm","egupux","clpeg","eysabqd","ftaabvd","nehiyz","llzvllp","opdbhxe","iktmt","jibxdia","kqvxbd","incitqc","ycmxpcu","iyvgzzw","crpdcj","sidsjh","kgezv","lcfmbqk","gcbfgyl","pfvwdw","xuokr","ttwxr","pwjyzqo","bzwwd","filtf","jgoqowg","fajtelw","shiipvq","hmvkgc","dyftrof","yqpfd","jsekqgo","pmzzg","dtvrwu","hjsmeh","evhhtfx","fvcjgvz","dbwpm","ukhfm","vmnraze","krofx","azixozl","hjxaf","vyhrrs","aoadmh","vkowji","vhibgvi","xvzesj","ponkak","emihkjk","csnszm","tgczmcq","rbsete","gecld","bhhzhf","bxrobn","stjazoo","wdxjqoz","tgpwp","vcwdrt","qudhlzk","ompwav","jmprpxl","glbqwbo","lgoskh","oggliik","wgyfy","xjdxw","zlrli","ovjpalp","qochtic","iokibyb","tfwrw","pnrvq","ydqki","qezwrg","jkmdez","fxgst","vwndsd","qdvixjc","oavco","bgyoqb","trhyic","fsedk","sqbtbic","cgmsc","hhbdwph","gwjugf","euyksf","zuqkbkz","qojte","juhhio","vbqns","qtqdwxq","cddjg","lhdonf","wenfim","edbnh","pgxhz","uiuibk","ffnebsh","fmiluhn","drjvl","jqeew","bfzxfbq","wodiszp","ccjdl","qsreonz","mpkgxnc","wvbvdt","iyucb","genuw","tkotnrf","jyosmeh","ptrejw","ysqduhw","qdhhpol","urtefdd","umjkiqb","fdmqwym","yhxrmwy","vqvemxw","ngyxej","jktbnfn","xwrlfh","grdjt","htvst","bwjqj","xkknjzx","urxzrhe","yleehv","qhnrnm","gxunm","nrcnjb","zengpfo","rgtea","tqtjvyq","kytru","yvezlk","eagaeu","gfcmykk","lpvorv","zrfqwln","kxogsw","caipbs","okmrred","xymdm","lgaqvdm","acfqs","xyqvmz","afeuamu","dyvnk","deneyc","vdrfa","cwowt","kdsln","flfdi","tjylmjq","wkjgv","grgok","wkxjrqc","rmehg","xwysywc","blbwsfe","pkdfb","sjkpmgp","kvpeo","ynmclu","kuipg","ndynlz","zpccm","vvufkf","arepauw","bhzatz","nltizba","deygc","rinuslx","wtdffat","haxth","syvto","nalhg","dweia","fwptgp","kyghi","trvkxj","tjjsi","txfwg","ckbjmhp","xxzwt","ibrrhzo","opqfxw","zdhts","inyxkz","yvzpnkv","vkfpyab","rluwgn","njkfzai","dczsp","txzguhg","zabgws","hqpze","rkiqxiy","znnhup","igbjvm","uzitf","tklalwq","qvnqrv","ozxez","zasuxek","shdesj","sqrmret","hhohwv","gmrpjyx","apbounv","ksrlmy","teijs","sbbwdkq","xfnepb","sdndv","bsqsu","wzwkxan","myxucw","kbootkn","ajuwe","kfagvn","egubnv","wpapn","kpyvqje","xqusvw","wdnlwi","ofrmtx","gufkhfp","uiepetm","hbmcyuz","znjudo","sxhaw","sfpkix","ionol","sdynwr","ogjrvu","hrviayo","rxqbzij","zqbqcnd","gnspr","aseaogj","jmilj","odzln","lkosnfn","lylie","xknvovh","tiwdt","nuubcbn","uuolcz","yphmubs","slwnmuy","sillo","tawzv","ceqhcq","yhwqc","weqosqx","zlpfc","pesou","qcoxhn","glukzp","slnntsr","vrnecb","tgpzc","vfphtw","hhrhb","tqqruq","wubkti","feoji","gvkfxbc","niefhs","laxctf","dkuvwrk","qvgyor","rzrcx","kgnbhma","ooxbos","nrvcbh","cygcgx","phailf","dhywuc","ghbtjve","zsbop","blwig","zjunnnt","lsorddc","xzxpsw","gdjbkqq","nnplvmo","ogbhh","ezzixgx","ulhcm","jyoib","tqsynj","afwde","yzlvsc","hqiktej","yjnrz","wnehhvu","iphfun","flemh","ekhpevl","vsuvifr","tkqsjgh","byifz","bdmac","rxniyex","uyfat","kqcaw","geslkc","yycxo","ozhwc","uhvrnu","cwqkhc","inyye","wugqx","neadky","thdrq","hjfvbu","ssiryz","pukibcp","ugwmef","dhepm","iuckeqa","rbmzqdo","vjqhp","juxxe","frwzmg","kmyiio","kujncqh","zkfwfu","ydfqud","llomsi","chgnxaf","xdbqnnm","gqefv","yxpaagh","hkhepx","zvjcv","cuwlqky","njsqc","ntwwugb","ueowqyv","nvjkvn","rwneyx","tunus","uzhiokr","jpqwmc","agjpf","kfkimwi","jwbzayv","hkpfezr","mrhcz","zsiyd","emlwr","rvtij","nzrklc","kjvfk","hvalv","opetw","xikypsh","qxzdgfk","eivwlk","jigfntu","xzlqrhe","havesdr","qjjuz","xaztruq","bjvbc","razbg","rhpeb","rmlxj","skczk","wkoln","wjhxp","wtjwbxb","utqcopa","zejkkqc","ahtphqk","kqljz","evqbxp","zroto","kzahb","fhfudq","acdbtxl","zbzwtki","qyokyvw","vzgfc","dpisna","vbtzi","myybnbo","cypekcl","lmzxhlc","imave","zljiv","jquqfpx","rovglf","hrgdiy","dbmqccb","oansv","rkbfupu","ckikn","okfjabc","lmzybam","dnguecw","suuzvg","jyrfcjf","gmbwpro","vijpx","ztzhae","evudpis","rhukk","zvvysqt","eeupgzf","miyrmy","otnppv","bxbmemi","urlgc","vlpeky","xyuvsnq","kmzblq","xlxeh","bvyzom","cvifd","mwpwqs","wiefi","estqwzk","ywwlxkw","ycjzq","abinmmi","wnsmhf","wpiovoj","pnpxj","tlakw","oowfldu","swwspkr","navvrtq","zisepnl","ykhgngv","ctpztqn","orjupt","qrikx","ipayuw","qghgpac","vylwfan","nxbrxox","hrmsyql","kgfvgm","drztnox","ylcaevd","dtnoogi","uloby","dliyylt","ofovbfe","lkadd","whlnxc","qzuwaex","jgiybi","lapvft","bvtbb","mmuicte","zxljjm","hbyadfs","dhhrmzw","rixxzcd","zrpsmjk","zukod","akfzy","uhghq","qysmbvg","wiiylz","ndhct","jjmosl","baxxcl","sdpxrhi","kijths","euuvlm","gzfdtmf","gisafmb","alrljrs","xzxtu","nzntjsh","ixzifh","jnemfn","mjwpmt","jjnsz","mrpvsh","zmymhr","fxajnk","fvchx","ctfen","unzkea","wxqshiv","fslggly","alhsv","ievdt","uszul","dqvnn","nubyaw","vuhtry","yvyuw","nzeakhi","aecewua","apinf","yuadv","rshsql","bfcekfg","nrnvgh","dospg","qjvafyy","akyzwc","wmzpeoo","kqbwkp","vhhdthp","ncide","gxbcz","kvgpl","folmcj","apcbmjk","pjqkid","bkymza","jjonu","jekzf","uphqb","lmtyj","gxncpbf","nhzxohy","zijdtlu","vwhuf","jrdrncs","fanyacg","elvtz","inpai","ijdhbn","espowa","uhsoqod","imtpbfx","azcgkdm","zxnabd","wcwgki","qsnxt","uxzxnn","rvvts","jthar","gqhea","ozknc","dzjfmaj","cqagwu","ekqtkn","offfk","nglyfcx","hbypi","uimrjdu","ioyldwm","mfxyoke","tvkmyy","aunkwfw","auvgf","otjwveh","ylcmhve","plamrd","azpse","lmgjuvz","wmwsr","yzely","tixzm","hmnkahu","efwej","apzqk","lddbe","obbynls","olbzsfe","flxmr","lymsm","hpggqgb","setvqk","zcozseg","lwtkzid","cesdgkz","ztcuk","aaloa","nntpx","wqwvz","hntvr","seieb","eesqi","cwika","vzyxk","agneot","brzui","dibidoa","zxtkh","bqjqx","rpcrpm","zabhpi","yhmdeax","xkredt","znqco","dvdktwc","wrdhzpv","pkgku","ggxiy","mopen","ztnojrt","zqggkwg","tzttopb","nfmil","ockwc","uhhnetx","mfuhqvj","jfdvft","fmtauaa","aecrbd","rbwcgbe","dwikan","opdqt","xjgepua","swlejnr","uxcyk","ksgba","htvjxd","pkslsr","kejien","vdfkja","epkmul","tavqu","ovetpuc","mzddfq","zenww","vqcmbu","jswxwuq","xwaiq","fjaob","damrury","jtgdbl","yyfwo","qbyysvr","zhyya","krhkn","xkajxza","gtergto","ryyncei","gvsjifv","ixzrjye","kydljfl","fomuwx","sogifn","aibvs","rexqzd","xxmmyi","lqowm","poqtrm","gguopb","ovslq","zhadl","iayhnl","bryuox","ftxsen","fkzld","digvjmy","tksisub","hsvtqyv","hwmwrvg","ygxrn","bnvfdr","yxsep","losnt","ufgfc","puvtt","mwgro","gochlx","zyxmjy","eqqlapu","xpnmt","lwpii","ycgyym","mfldyej","xrwia","lkrhqpa","gpdjf","qeigd","vxhhz","lqfog","kamqj","qojpju","uazbq","lvjnknp","ufonf","jkulf","arnisqj","agmimzz","gxxrxv","otmmjeh","grbhwt","slgzfri","xifzy","wpjcedd","qppjxcp","xwpqam","qdavyd","tjtjqip","yekkcl","wppwo","bypmwu","buelsm","gjfpqaz","qjyun","umgwwfx","bwztww","gqztmc","gwkau","clvwm","bplcyhx","viuujti","ddsnnmu","uoopp","hnild","xxucpn","lefbl","fmshwno","dnynva","vcczycw","hmbuk","ktzwykq","zoaboh","lngbjer","vpapqwk","tjxknx","rnxjtxa","vuqqpfs","uckomez","kffasw","xbprf","dplkl","uxqwg","mjtia","cptmyyc","bciou","cuthc","uhapaz","wqrukop","dpxnu","iwkibv","pqjzg","pddsg","ipyqfx","feilkrw","gcgcmr","smmii","mcsmgcl","whxrls","ytgvm","qpmewww","puzbgps","oeademr","pscohn","dfnizil","uzkrnxw","jftmnky","csxqt","rtwvb","nyyzq","gissdh","ubquvg","njpzndq","ozmmj","auveib","yepmcyx","oxucf","apfdom","exfdx","gbeqrm","qgsin","kvhzjqt","sgutjsl","iqdmzs","zumvzzb","flqlcdv","mwrkxtn","hxbyh","vajrd","uwzok","ngfppvv","oiuaa","vbgdwkx","oenhor","itpsqpy","nsyvueo","shsdh","ezzftc","zgjzjd","tfrswc","fagjje","ggcrtkd","bqjpg","vrnparn","nghqf","pwhth","qiosj","oqtmob","sjcjf","aiiya","yeufwjq","kaxfdge","szrtsh","sjnveq","akmkvww","ydcghrx","rdphx","iqyse","frugy","uozwd","dobvnp","gfsbdy","kvkqi","jvazbl","enlse","nyyfq","qnahoma","izttww","gvbrgm","cwxslml","aaayqqy","izsirw","gbvkhy","csluwg","zlekmab","zgdhufz","dpdkfs","tyiltm","nbfsxq","gpkqxud","zcfpo","ivfukc","mgeuipg","xcqjjxe","wymfs","jqtgv","fjtpm","isxbtp","lipwgm","stggw","jucrd","koidvz","ojsbaz","hvqzbwe","pgrnmff","coeijd","bsyebu","cykana","bkpkb","wdgrma","sooev","mjrzv","roxgv","uuqmaxt","whwwfsv","easwcx","iwynz","djwjz","idmfgu","ruyovq","qkrra","nnbtzzn","rhzdau","vaaenou","ghmvsr","ifazg","zcfqi","snkwuj","wbojg","acyeou","mnekm","yluki","szbynf","gkpmwp","ehhcbqv","bznqkop","hmznubr","lhtbzor","fjgghr","twpra","ckmgq","fyejeyj","ljhbve","nhilk","qqcrwil","yccvy","bykbo","qrjzh","sfircwc","kaqbfah","hssvaq","gxfemhe","pnoxjyi","epvmt","kpatr","dkadhfs","smsrwky","gcyaoip","pwrirte","xksyb","tbnnzb","gikkibk","xuptbzm","hohjr","sxibnhg","idotf","wlquo","getxtl","vtoxjz","pduuq","nozqw","qohkd","yfpocgp","jsdulbr","iznjxv","lrjje","qvraemw","zszjl","tihbwz","hpifw","aeijnyc","acftqln","wewaoge","kifbzq","wnjgqd","htqyju","plowdsv","ojmgjqs","mlnipcn","qhxmby","dtmvf","lepkuoi","gbfotit","heuvsqv","bpuzsa","lsmirj","qxehdj","lsqhcah","lavemk","nijtshc","clwpxm","wmckhqf","bmcuf","pjyif","mnzjsh","znjunai","irlhur","bxgfr","lrkki","zaqesb","gdjrzta","axkxacw","myoulrl","jczzn","sdwutee","mmvzpul","ungobhh","mpjzrv","ovwstq","ajyiy","bybxqc","adpec"])

14018

## 541. Reverse String II

先找规律
1. `s[0 * k: 1 * k]`颠倒，`s[1 * k: 2 * k]`不变
1. `s[2 * k: 3 * k]`颠倒，`s[3 * k: 4 * k]`不变
1. `s[4 * k: 5 * k]`颠倒，`s[5 * k: 6 * k]`不变
1. ...
1. `s[2 * i * k: (2 * i + 1) * k]`颠倒，`s[(2 * i + 1) * k: (2 * i + 2) * k]`不变

In [42]:
class Solution:
    def reverseStr(self, s, k):
        """
        :type s: str
        :type k: int
        :rtype: str
        """
        return "".join(s[2 * i * k: (2 * i + 1) * k][:: -1] + s[(2 * i + 1) * k: (2 * i + 2) * k] for i in range(0, len(s) // (2 * k) + 1))

In [41]:
Solution().reverseStr("abcdefghijk", 2)

'bacdfeghjik'

## 925. Long Pressed Name

一开始想用re。

In [35]:
import re
class Solution:
    def isLongPressedName(self, name, typed):
        """
        :type name: str
        :type typed: str
        :rtype: bool
        """
        if len(name) > len(typed):
            return False
        return bool(re.match(name.replace("", "+")[1: ], typed))

没想到会超时。

仔细一想这道题，有点像[392. Is Subsequence](#392.-Is-Subsequence)，但是更严格，不允许母数列中子数列项之间有和两边项不同的元素出现。

In [35]:
class Solution:
    def isLongPressedName(self, name, typed):
        """
        :type name: str
        :type typed: str
        :rtype: bool
        """
        if len(name) > len(typed):
            return False

        

In [39]:
Solution().isLongPressedName(name = "alex", typed = "aaleex")

True

In [40]:
Solution().isLongPressedName(name = "saeed", typed = "ssaaedd")

False

## 929. Unique Email Addresses

In [63]:
class Solution:
    def numUniqueEmails(self, emails):
        """
        :type emails: List[str]
        :rtype: int
        """
        return len(set(i.split("@")[0].split("+")[0].replace(".", "") + i.split("@")[-1] for i in emails))

In [64]:
Solution().numUniqueEmails(["test.email+alex@leetcode.com","test.e.mail+bob.cathy@leetcode.com","testemail+david@lee.tcode.com"])

2

## 905. Sort Array By Parity

In [65]:
class Solution:
    def sortArrayByParity(self, A):
        """
        :type A: List[int]
        :rtype: List[int]
        """
        evens = []
        odds = []
        for i in A:
            if i % 2 == 0:
                evens.append(i)
            else:
                odds.append(i)
        return evens + odds

In [65]:
class Solution:
    def sortArrayByParity(self, A):
        """
        :type A: List[int]
        :rtype: List[int]
        """
        return [i for i in A if i % 2 == 0] + [i for i in A if i % 2 != 0]

## 922. Sort Array By Parity II

In [3]:
class Solution:
    def sortArrayByParityII(self, A):
        """
        :type A: List[int]
        :rtype: List[int]
        """
        evens = []
        odds = []
        for i in A:
            if i % 2 == 0:
                evens.append(i)
            else:
                odds.append(i)
        res = []
        for i in zip(odds, evens):
            res.append(i[1])
            res.append(i[0])
        return res

但是老是append很慢。

In [7]:
class Solution:
    def sortArrayByParityII(self, A):
        """
        :type A: List[int]
        :rtype: List[int]
        """
        res = A.copy()
        oddIndex = 1
        evenIndex = 0
        for i in A:
            if i % 2 == 0:
                res[evenIndex] = i
                evenIndex += 2
            else:
                res[oddIndex] = i
                oddIndex += 2
        return res

照理说应该比append快……但是居然比前面的慢。

In [8]:
Solution().sortArrayByParityII([4,2,5,7])

[4, 5, 2, 7]

## 883. Projection Area of 3D Shapes

要求把左视图、前视图、顶视图的面积加起来。

感觉用Numpy会超级方便。

In [13]:
import numpy as np
class Solution:
    def projectionArea(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        grid = np.array(grid)
        
        return np.sum(grid != 0) + np.sum(grid.max(axis = 0)) + np.sum(grid.max(axis = 1))

In [14]:
Solution().projectionArea([[1,0],[0,2]])

8

In [15]:
Solution().projectionArea([[1,1,1],[1,0,1],[1,1,1]])

14

In [16]:
Solution().projectionArea([[2]])

5

但是leetcode不知道怎么回事，报runtime错误。

那再用纯python写一下吧。

In [23]:
class Solution:
    def projectionArea(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        top = sum(sum(col != 0 for col in row) for row in grid)
        front = sum(max(row) for row in grid)
        left = sum(max(row) for row in list(map(list, zip(*grid))))
        return top + front + left

In [24]:
Solution().projectionArea([[1,0],[0,2]])

8

In [25]:
Solution().projectionArea([[1,1,1],[1,0,1],[1,1,1]])

14

In [26]:
Solution().projectionArea([[2]])

5

看到最快的写法还可以优化

- top的面积可以直接宽平方、再减去高度是0的格子的个数
- front的面积我那种已经是最快了
- left的面积我是先转置、再逐行找最大值，已经是最快了

一开始我以为是他用了炫酷的functional style写法才这么快的，但是自己测了一下发现速度和list comprehension没区别。

In [29]:
class Solution:
    def projectionArea(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        top = len(grid) ** 2 - sum(row.count(0) for row in grid)
#         front = sum(max(row) for row in grid)
        front = sum(map(max, grid))
#         left = sum(max(row) for row in map(list, zip(*grid)))
        left = sum(map(max, map(list, zip(*grid))))
        return top + front + left

In [30]:
Solution().projectionArea([[1,1,1],[1,0,1],[1,1,1]])

14

## 892. Surface Area of 3D Shapes

这次要数表面积。

- 上表面和下表面积相等，最好数
- 4个侧面积有点难数。其实也不难，要2个方向，来回算差分、再求和

In [36]:
class Solution:
    def surfaceArea(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        top = len(grid) ** 2 - sum(row.count(0) for row in grid)
        bottom = top
        left = sum(row[0] + sum(max(row[i] - row[i - 1], 0) for i in range(1, len(grid))) for row in grid)
        right = sum(row[:: -1][0] + sum(max(row[:: -1][i] - row[:: -1][i - 1], 0) for i in range(1, len(grid))) for row in grid)
        
        transposed = list(map(list, zip(*grid)))
        front = sum(row[0] + sum(max(row[i] - row[i - 1], 0) for i in range(1, len(grid))) for row in transposed)
        back = sum(row[:: -1][0] + sum(max(row[:: -1][i] - row[:: -1][i - 1], 0) for i in range(1, len(grid))) for row in transposed)
        return top + bottom + left + right + front + back

In [38]:
Solution().surfaceArea([
    [0, 1],
    [2, 0]
])

16

In [39]:
Solution().surfaceArea([[2]])

10

## 893. Groups of Special-Equivalent Strings

一开始的想法是，用Counter把每个字符串的奇下标、偶下标做两个直方图出来，作为这个字符串的**特征。**那么特征相同的字符串直接就看出来了，然后再用一下set，过滤掉特征重复的字符串。

In [68]:
import collections
class Solution:
    def numSpecialEquivGroups(self, A):
        """
        :type A: List[str]
        :rtype: int
        """
        features = [(frozenset(collections.Counter(i[:: 2]).items()), frozenset(collections.Counter(i[1 :: 2]).items())) for i in A]
#         print(features)
#         print(set(features))
        return len(set(features))

但是挺慢的……

那么来想一下怎样存、比较特征更快。上面的做法是用Counter，因为Counter不是hashable，所以没法放在set里面，只能先把Counter变成hashable的frozenset。算是最容易想到的做法吧。但是真的很慢。

答案上最快的做法是取出所有奇下标字符、排序，再取出所有偶下标字符、排序，拼起来，这样把字符串转换成它的特征。再全部放到set里面。

In [81]:
import collections
class Solution:
    def numSpecialEquivGroups(self, A):
        """
        :type A: List[str]
        :rtype: int
        """
        return len(set("".join(sorted(i[:: 2]) + sorted(i[1:: 2])) for i in A))

In [82]:
Solution().numSpecialEquivGroups(["abc","acb","bac","bca","cab","cba"])

3

In [83]:
Solution().numSpecialEquivGroups(["aa","bb","ab","ba"])

4

## 908. Smallest Range I

看了好多遍问题没有看懂什么意思。

In [84]:
class Solution:
    def smallestRangeI(self, A, K):
        """
        :type A: List[int]
        :type K: int
        :rtype: int
        """
        maximum = max(A)
        minimum = min(A)
        if maximum - minimum > 2 * K:
            return maximum - minimum - 2 * K
        else:
            return 0

In [90]:
class Solution:
    def smallestRangeI(self, A, K):
        """
        :type A: List[int]
        :type K: int
        :rtype: int
        """
        return max(0, max(A) - min(A) - 2 * K)

In [91]:
Solution().smallestRangeI([1, 3, 6], 3)

0

In [92]:
Solution().smallestRangeI([0, 10], 2)

6